# Data Wrangling with `pandas`

```{contents} Table of Contents
:depth: 4
```

## Introduction: Why It's So Important to Get Data Management Right
### The Case of "Genomic Signatures to Guide the Use of Chemotherapeutics"
In 2006, a study "[Genomic Signatures to Guide the Use of Chemotherapeutics](https://www.nature.com/articles/nm1491)" was published in *Nature Medicine*. This study tackles the question of why chemotherapy works well for some cancer patients and less well for others. The authors claimed that they could predict whether and how treatments would work for a patient based on the sequence of chemicals in the DNA in the patient's cancer cells. This study got a lot of attention as a major breakthrough in cancer treatments. The hope was to use genomic data to tailor a treatment regime for individual patients that maximizes the likelihod that the treatment would be effective. 

Unfortunately, as you can see on the journal's website for this article, the study has been retracted. The story of how this study came into question, and the damage it did before it was retracted, is described by Keith Baggerly in his talk "[The Importance of Reproducibility in High-Throughput Biology: Case Studies in Forensic Bioinformatics](https://www.youtube.com/watch?v=7gYIs7uYbMo)":

In [1]:
from IPython.display import IFrame
IFrame(src="https://www.youtube.com/embed/7gYIs7uYbMo", width="560", height="315")

The slides for this talk are available [here](http://www.bioconductor.org/help/course-materials/2009/BioC2009/talks/wrap_bioc_shareb.pdf). In short, Baggerly and his coauthor Kevin R. Coombes attempted to reproduce the findings in the Potti et al study, and they were unable to do so. There is a difference between reproduction and replication: reproducing a study involves using the same data used in the original study and writing code that yields the exact same results; replication involves repeating the study using new data. While replication is an essential practice for establishing the validity of a study, replication is also a higher standard than reproduction. If a study cannot be reproduced, then there must have been errors in the process collecting and cleaning the raw data that led to the publication of incorrect results.

After deciding to replicate the Potti et al article, Baggerly and Coombes were able to get a copy of the raw data that were used in the study, but they did not find scripts or documentation that described the steps the original study took to clean the raw data. That's because there were no scripts and no documentation: the data were cleaned on the fly, probably with a point-and-click interface like Microsoft Excel, with no records of what was done to the data.

Without having good documentation available that explains how research was done, Baggerly and Coombes had to resort to "forensics". They started with the results and the raw data, and they inferred what the researchers must have done to obtain the reported results from the raw data. They were alerted to potential problems with the study when they tried to replicate the finding that there is a clear difference between two types of drugs, as shown in the graphic on slide 13 of Baggerly's slide deck. It should have been an easy finding to replicate. Instead they found significant overlap between the two drugs, as shown in the graphic on slide 14. They responded by trying to reproduce exactly what was done to the raw data in the original study to produce the first finding. After exhausting all possible *valid* data management steps, they started **making mistakes on purpose** to see if one of those mistakes would replicate the published results. 

Baggerly and Coombes found that the authors of the original paper made an error in which the codes for each reported gene were off by one. That is, the data for each gene is matched to the label for the gene whose data exists one row above. This error happened, according to Baggerly's best guess, because the software that the authors used requires the inputted data file to NOT have a header row for column names. But they seemed to just copy and paste the data, along with the header row. A second result that Baggerly says should "bother you" is that, on the heat map on slide 29, Baggery and Coombs matched 6 of the 7 heat maps despite the fact that they only matched the reported gene names for 3 of the 7 drugs. Therefore they would expect to reproduce the heat map for only these 3 drugs. Baggerly doesn't know why they matched the heat maps for the other 3 drugs despite the fact that they are using the wrong data.

A second discrepancy that raises a red flag for Baggerly is the plot showing the sensitive and resistant patients, as compared to the analogous plot from the paper that introduced the test dataset. The plot shows 13 sensitive and 11 resistant samples, whereas the paper that introduced the data shows 11 sensitive and 13 resistant samples. Baggery believes that the authors mixed up the 0 and 1, indicating sensitive and resistant, in the spreadsheet.

In short, Baggerly and Coombes were only able to reproduce the findings in the Potti et al study by making "small" mistakes on purpose. When I say small, I mean that these sorts of errors can occur frequently when using a point-and-click interface for data management and a copy-and-paste method for transfering data. These errors also occur without informing the user of any errors. But in practice these small errors have profound impacts on the results of the study. If those results are taken seriously and applied outside of the specific study, these mistakes can do a lot of harm in the world.

In the case of the Potti et al study, while Baggerly and Coombes were investigating the paper,
drug trials began at Duke University based on the results of the study. The treatment was administered to the patients with childhood leukemia. Because of the mistake that swaped the 0s and 1s, the treatment was administered to patients who are resistant to it, will not benefit from it, and may in fact be harmed by it.

Baggerly and Coombes wrote a report detailing these inconsistencies and a few others - there's evidence that some samples were reused 2, 3, or 4 times, and 4 genes appear in the results despite the fact that they are not included in the test data - and sent the report to *Nature Medicine*. Their report caused Duke to stop the clinical trials of these drugs pending an investigation. But a short while later the trials were restarted, with the statement that the investigation had "strengthened confidence in this approach." The investigators refused to share their findings. Baggery and Coombs has to issue a freedom of informatiom act (FOIA) request to see the results of the investigation, only to find that no mention was made of the problematic drugs. Writing directly to Duke yielded no response. They had to get many prominent biostatisticians to sign a joint letter of concern. The letter got some attention in the media, but mostly because it was revealed that the principal investigator of the flawed studies lied on his CV about being a Rhodes scholar! Only after the letter generated a serious amount of negative media attention for Duke did the trials get suspended.

### The Reproducible Research Movement
There are four lessons we can learn from that Baggerly and Coombes's work to reassess the Potti et al. First, mistakes at the data management stage happen all the time, and when the environment uses a point-and-click interface the errors are easier to miss because of a lack of a sufficient error system. Second, small errors can have big effects. It's easy to confuse categories when they are labeled with arbitrary numbers instead of words, but if that confusion results in mixing up the treatment and control groups then every effect reported in the study will have the incorrect sign, and all of the conclusions we draw will be the exact opposite of the truth. Third, it is common for people who do work with data to fail to keep records of their code and the steps they used to manipulate the raw data prior to an analysis, and without documentation, it can be extremely difficult to impossible to catch errors in the data management process. Fourth, while most scientists are hopefully open to seeing and admitting their mistakes, there aren't a lot of incentives in academia and other fields to make the research stream more transparent.

In "[The Reproducible Research Movement in Statistics](http://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.667.6492)", Victoria Stodden describes the problem:

> The research system must offer rewards for reproducible research at every level from departmental decisions to grant funding and journal publication, incorporating notions of code and data sharing into institutional 
promotion and hiring and grant proposal review. The current academic research system places the primary emphasis on publication and little on reproducibility. This has the effect of penalizing those researchers that produce reproducible computational research. Software development has been characterized as support of science rather than doing real science. The result is that scientists are discouraged from spending time writing, testing, or releasing code. With the ever-increasing pervasiveness of computation and software across the research landscape, such attitudes and practices must change (p.1069-1070).

As a result of these issues, the last decade has seen a remarkable cultural shift in fields that use data towards making data management easier and more transparent. These efforts are often called part of the "reproducible research movement". The movement involves two efforts: first, a greater push to develop software that makes it easier to clean data and to share and communicate code; second, a drive to change standards of best practice in fields that use data. Since Victoria Stodden's article in 2013, data science has incorporated both parts of the reproducible research movement. In Python, the `pandas` package makes it much easier to clean data using code that is intuitive and relatively easy to follow, and Jupyter notebooks combine code, results, and text that explains the code in one document. Modern data science requires practioners to share or be ready to share their code, often in a notebook.

### The Origin of `pandas`
The `pandas` package in Python is written and maintained by Wes McKinney, who also wrote [Python for Data Analysis](https://learning.oreilly.com/library/view/python-for-data/9781491957653/). On the `pandas` [team page](https://pandas.pydata.org/about/team.html), under governance, McKinney is listed as the `pandas` "[Benevolent Dictator for Life](https://en.wikipedia.org/wiki/Benevolent_dictator_for_life)". The phrase "Benevolent Dictator for Life" is given to leaders of important open-source projects, and originated with Guido van Rossum, the creator of Python.

McKinney developed `pandas` in 2008 and first released the project publicly in 2009. Though not explicitly presented as a contribution to the reproducible research movement, the goal of `pandas` was to replace existing tools for data manipulation that had led to major mistakes in research, as was the case for the Potti et al article. According to this [profile of Wes McKinney](https://qz.com/1126615/the-story-of-the-most-important-tool-in-data-science/) in Quartz Magazine, `pandas` "makes it so that data analysis tasks that would have taken 50 complex lines of code in the past now only take 5 simple lines, because McKinney already did the heavy lifting." Specifically:

> Python was missing some key features that would make it a good language for data analysis. For example, it was challenging to import CSV files (one of the most common formats for storing datasets). It also didn’t have an intuitive way of dealing with spreadsheet-like datasets with rows and columns, or a simple way to create a new column based on existing columns.

> Pandas addressed these problems. David Robinson, a data scientist at Stack Overflow, explained the importance of it in technical terms. “The idea of treating in memory data like you would a SQL table is incredibly powerful,” he says. “By introducing the ‘DataFrame,’ Pandas made it possible to do intuitive analysis and exploration in Python that wasn’t possible in other languages like Java. And is still not possible.”

That is, `pandas`, like SQL, is a declarative language, not a procedural one. Simple functions and methods in `pandas` call underlying imperative code that performs the requested action by operating on vectors, matrices, and arrays. `pandas` as much as possible uses `numpy` functions to perform actions as quickly as possible (by using `numpy`'s [vectorized](https://en.wikipedia.org/wiki/Vectorization_(mathematics)) code). In short, the motivation for the development of `pandas` is to make data manipulation as straightforward as possible and for those functions to run as quickly as possible.

`pandas` is an extremely popular package for data science, and the Quartz article asserts that much of the growth in popularity that Python has experienced on Stack Overflow can be attibuted to `pandas`. While `pandas` is currently the [34th most frequently downloaded Python package](https://hugovk.github.io/top-pypi-packages/), it is certainly one of the most frequently downloaded for data science applications in Python.

### Tidy Data
In Module 7, we discussed the concept of Tidy Data as described by Hadley Wickham, founder of RStudio and creator of the `tidyverse` packages in R, in the article "[Tidy Data](https://www.jstatsoft.org/article/view/v059i10)". Tidy Data is an organizational schema for data that makes most kinds of data analysis possible. To qualify as tidy, data must follow these rules:

> 1. Each [feature] forms a column.
> 2. Each observation forms a row.
> 3. Each type of observational unit forms a table (p. 2).

In addition, even when data are arranged in a single table in which records are rows, features are columns, and all records are of like units, there are many steps a researcher should take to manipulate the data to make a dataframe easier to work with. Hadley Wickham defines four essential "verbs" of data manipulation:

> * Filter: subsetting or removing observations based on some condition.
> * Transform: adding or modifying variables. These modifications can involve either a single variable (e.g., log-transformation), or multiple variables (e.g., computing density from weight and volume).
> * Aggregate: collapsing multiple values into a single value (e.g., by summing or taking means).
> * Sort: changing the order of observations (p. 13).

This philosophy of data management is built into the `tidyverse` packages for R, but it also forms a guiding principle for data manipulation with `pandas` because `pandas` includes tools to accomplish all of these tasks. In fact, Hadley Wickham and Wes McKinney are currently working together on a project called [https://ursalabs.org/](https://ursalabs.org/), which aims to bridge the R-Python divide and to develop platform-independent data science tools.

## Example: The American National Election Study
The [American National Election Study](https://electionstudies.org) (ANES) is a massive public opinion survey conducted after every national election. It is one of the greatest sources of data available about the voting population of the United States. It contains far more information than a typical public opinion poll. Iterations of the survey contain thousands of features from thousands of respondents, and examines people's attitudes on the election, the candidates, the parties, it collects massive amounts of demographic information and other characteristics from voters, and it records people's opinions on a myriad of political and social issues.

Prior to each election the ANES conducts a "pilot study" that asks many of the questions that will be asked on the post-election survey. The idea is to capture a snapshot of the American electorate prior to the election and to get a sense of how the survey instrument is working so that adjustments can be made in time. Here we will work with the [2019 ANES pilot data](https://electionstudies.org/data-center/2019-pilot-study/). To understand the features and the values used to code responses, the data have an associated [questionnaire](https://electionstudies.org/wp-content/uploads/2020/02/anes_pilot_2019_questionnaire.pdf) and [codebook](https://electionstudies.org/wp-content/uploads/2020/02/anes_pilot_2019_userguidecodebook.pdf). The pilot data were collected in December 2019 and contain 900 features collected from 3,165 respondents. 

Despite a great dfeal of pre-processing on the part of the data authors, the data is not currently in form we can analyze. We will use the tools available in `pandas` to manipulate the data and prepare it for analysis. The data are available in CSV, SAS, and Stata formats. I put the CSV version on my Github repository so that we can have easier access to it, but I encourage you to access the data yourself by registering for a free account on electionstudies.org. 

We start by loading the `numpy` and `pandas` packages, along with some additional packages and modules to perform some specialized tasks. Please use `pip install` or `conda install` to install these packages if you have not yet done so:

In [2]:
import numpy as np
import pandas as pd
import sidetable
import warnings
warnings.filterwarnings('ignore')

Then we load the data with `pd.read_csv()`:

In [3]:
anes = pd.read_csv("https://github.com/jkropko/DS-6001/raw/master/localdata/anes_pilot_2019.csv")

These data are technically in tidy format because the records are stored in the rows, the features are stored in the columns, and the rows all represent individual people. But there are still problems that will keep us from being able to use the data in analyses. There are more features here than we can manage in a feasible amount of time. Many of the features have names that we might want to change to be more descriptive, and use arbitrary numeric codes to stand in for categories. If we look at the codebook, we will also see that negative numbers in the cells represent missing datapoints, and unless we take steps to replace these datapoints they will be read as valid numeric data. We will use the tools in `pandas` to manipulate the data so that we keep only the features of interest, labels throughout the data are intuitive and readable, columns have the correct data type, and missing datapoints are coded as missing.

## Tools for Understanding the Data and Diagnosing Problems
Before we can start manipulating the data, we need to be able to effectively look at the data to catch the problems that we need to address. All of these tools exist as methods or attributes of a `pandas` dataframe or of a specific column of a dataframe. This discussion reviews some of the material covered in Module 2, because these methods are useful for assessing both whether a data file loaded properly and for seeing the problems that still exist after the data have been loaded.

First, to see the dimensions of a dataframe and the dataframe's memory usage, we can use the `.info()` method:

In [4]:
anes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3165 entries, 0 to 3164
Columns: 900 entries, version to flag_state
dtypes: float64(223), int64(600), object(77)
memory usage: 21.7+ MB


In this case, the `anes` data has 3,165 rows and 900 columns, and takes up a little bit more than 21.7 MB of memory. The `.info()` output also tells us that of the 900 columns, 223 have float (numeric with decimals) valued data, 600 have integer valued data, and 77 have object values, which refers to strings or categories.

Viewing the data table itself can be extremely useful. As we discussed in module 2, however, the default behavior limits the number of rows and columns that can be displayed in a dataframe. I find it useful to turn off the limit on the number of columns that can be displayed:

In [5]:
pd.set_option('display.max_columns', None) 

Now we can see all of the columns in any dataframe we call. With 900 columns, there will be quite a lot of sidescrolling, and it will be difficult to find any one column in particular. Viewing the data will become more useful once we limit the columns we will include in the data. To see the first few rows, use the `.head(n)` method, where `n` is the number of rows from the top to display. The first three rows, for example, are

In [6]:
anes.head(3)

,version,caseid,weight,weight_spss,form,follow,reg1a,reg1b,liveurban,youthurban,placeid1a,placeid1a_t,placeid1b,placeidimport,turnout16a,turnout16a1,turnout16b,turnout16c,vote16,turnout18a,turnout18a1,particip_1,particip_2,particip_3,particip_4,particip_5,particip_6,particip_7,particip_8,particip_9,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,vote20dem,vote20cand,vote20cand2,electable,vote20jb,vote20ew,vote20bs,cvote2020,tsplit1,contact1a,contact1b,contact2a,contact2b,contact3,contact4m_1,contact4m_2,contact4m_3,apppres5,frnpres5,immpres5,econpres5,apppres7,frnpres7,immpres7,econpres7,mip,econnow,finworry,confecon,improve1,national1,national2,conspire1,conspire2,conspire3,taxecon,billtax,trade1,trade2,trade3,trade4,richpoor2,guarinc,lcself,lcd,lcr,pop1,pop2,pop3,corrupt,immignum,refugees,dreamstr,dreamer,dreamstr1,dreamstr2,wall5,wall7,pathway,preturn,popen,release1,release2,famsep,tchina,trussia,tiran,tmexico,tjapan,hlthcare1,hlthcare2,abortion1,abortion2,freecol,loans,diversity5,diversity7,language,buyback,gw1,gw2,knowopioid1,knowopioid2,opioiddo,sentence,prek,demo4,experts,science,exphelp,elite1,elite2,elite3,elite4,ukraine1,ukraine2,excessive,rural1alt1,rural1alt2,rural2alt1,rural2alt2,rural3alt1,rural3alt2,rural4alt1,rural4alt2,conf_unemp,unemp,conf_interfere,interfere,conf_autism,autism1,autism2,conf_gmo,gmo1,gmo2,conf_warm,warm,conf_illegal,illegal,impeach1,impeach2,pk_cjus,pk_cjus_correct,pk_germ,pk_germ_correct,pk_sen,pk_spend,pk_geer,cheat,pid7x,pid1d,pid2d,pid1r,pid2r,pidstr,pidlean,ngun,shooting,dem_activduty,milyears,milyr1,milyr2,milyr3,milyr4,milyr5,milyr6,combat,harass1a,harass1b,rr1,rr2,rr3,rr4,health1a,health1b,hospital,feet,inches,nweight,bmi,disable1,disable2,disable3,disable4,disable5,disable6,smoker1,smoker2,exercise,relig1a,relig1a_txt,relig2a,att1,att2,att3,relig1b,relig1b_t,relig2b,relig3b,relig3b_t,attother,exptravel_ever,exphomesch,expfarm,expffood,expconvert,expholiday,explie,expshark,expdivorce,exparrest,expoverdose,expdefault,exppubasst,exphybrid,expmistake,explightning,exptravel_year,expindian,exphunt,expflag,exppublib,explottery,expshoponline,exppubtrans,expfight,expavoid,expknowimmig,expknowtrans,expbuyusa,expretire,expcolldebt,expknowpris,socmed_1,socmed_2,socmed_3,socmed_4,socmed_5,socmed_6,socmed_7,socmed_8,socmed_9,socmed_t,facebook1,facebook2,facebook3,twitter1,twitter2,twitter3,instagram1,instagram2,instagram3,reddit1,reddit2,reddit3,youtube1,youtube2,youtube3,snapchat1,snapchat2,snapchat3,tiktok1,tiktok2,tiktok3,raceid,racework,whitejob,race_sub1,race_sub2,voterid1,voterid2,serious,photo1,photo2,photo3,photo4,reinterview,birthyr,gender,educ,marstat,child18,race,employ,employ_t,faminc_new,votereg,ideo5,pid7,newsint,presvote16post,presvote16post_t,pew_bornagain,pew_religimp,pew_churatd,pew_prayer,religpew,religpew_t,religpew_protestant,religpew_protestant_t,inputstate,zipCode,FIPCounty,region,EnrollmentDate,CompletedSurveys,qualityControl_overall_scale,tsmart_P2012,tsmart_G2012,tsmart_P2016,tsmart_G2016,tsmart_P2018,tsmart_G2018,abortion1_skp,abortion2_skp,apppres5_skp,apppres7_skp,att1_skp,att2_skp,att3_skp,attother_skp,autism1_skp,autism2_skp,billtax_skp,buyback_skp,cexp1_grid_skp,cexp2_grid_skp,cheat_skp,combat_skp,confecon_skp,conspire1_skp,conspire2_skp,conspire3_skp,contact1a_skp,contact1b_skp,contact2a_skp,contact2b_skp,contact3_skp,corrupt_skp,cvote2020_skp,dem_activduty_skp,demo4_skp,disable_grid_skp,diversity5_skp,diversity7_skp,dreamer_skp,econnow_skp,econpres5_skp,econpres7_skp,electable_skp,elite1_skp,elite2_skp,elite3_skp,elite4_skp,excessive_skp,exercise_skp,exp1_grid_skp,exp2_grid_skp,experts_skp,exphelp_skp,facebook1_skp,facebook2_skp,facebook3_skp,finworry_skp,follow_skp,freecol_skp,frnpres5_skp,frnpres7_skp,ftasian_skp,ftb

To see the last several rows, use the `.tail(m)` method where `m` is the number of rows from the bottom. The last three rows are:

In [7]:
anes.tail(3)

,version,caseid,weight,weight_spss,form,follow,reg1a,reg1b,liveurban,youthurban,placeid1a,placeid1a_t,placeid1b,placeidimport,turnout16a,turnout16a1,turnout16b,turnout16c,vote16,turnout18a,turnout18a1,particip_1,particip_2,particip_3,particip_4,particip_5,particip_6,particip_7,particip_8,particip_9,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,vote20dem,vote20cand,vote20cand2,electable,vote20jb,vote20ew,vote20bs,cvote2020,tsplit1,contact1a,contact1b,contact2a,contact2b,contact3,contact4m_1,contact4m_2,contact4m_3,apppres5,frnpres5,immpres5,econpres5,apppres7,frnpres7,immpres7,econpres7,mip,econnow,finworry,confecon,improve1,national1,national2,conspire1,conspire2,conspire3,taxecon,billtax,trade1,trade2,trade3,trade4,richpoor2,guarinc,lcself,lcd,lcr,pop1,pop2,pop3,corrupt,immignum,refugees,dreamstr,dreamer,dreamstr1,dreamstr2,wall5,wall7,pathway,preturn,popen,release1,release2,famsep,tchina,trussia,tiran,tmexico,tjapan,hlthcare1,hlthcare2,abortion1,abortion2,freecol,loans,diversity5,diversity7,language,buyback,gw1,gw2,knowopioid1,knowopioid2,opioiddo,sentence,prek,demo4,experts,science,exphelp,elite1,elite2,elite3,elite4,ukraine1,ukraine2,excessive,rural1alt1,rural1alt2,rural2alt1,rural2alt2,rural3alt1,rural3alt2,rural4alt1,rural4alt2,conf_unemp,unemp,conf_interfere,interfere,conf_autism,autism1,autism2,conf_gmo,gmo1,gmo2,conf_warm,warm,conf_illegal,illegal,impeach1,impeach2,pk_cjus,pk_cjus_correct,pk_germ,pk_germ_correct,pk_sen,pk_spend,pk_geer,cheat,pid7x,pid1d,pid2d,pid1r,pid2r,pidstr,pidlean,ngun,shooting,dem_activduty,milyears,milyr1,milyr2,milyr3,milyr4,milyr5,milyr6,combat,harass1a,harass1b,rr1,rr2,rr3,rr4,health1a,health1b,hospital,feet,inches,nweight,bmi,disable1,disable2,disable3,disable4,disable5,disable6,smoker1,smoker2,exercise,relig1a,relig1a_txt,relig2a,att1,att2,att3,relig1b,relig1b_t,relig2b,relig3b,relig3b_t,attother,exptravel_ever,exphomesch,expfarm,expffood,expconvert,expholiday,explie,expshark,expdivorce,exparrest,expoverdose,expdefault,exppubasst,exphybrid,expmistake,explightning,exptravel_year,expindian,exphunt,expflag,exppublib,explottery,expshoponline,exppubtrans,expfight,expavoid,expknowimmig,expknowtrans,expbuyusa,expretire,expcolldebt,expknowpris,socmed_1,socmed_2,socmed_3,socmed_4,socmed_5,socmed_6,socmed_7,socmed_8,socmed_9,socmed_t,facebook1,facebook2,facebook3,twitter1,twitter2,twitter3,instagram1,instagram2,instagram3,reddit1,reddit2,reddit3,youtube1,youtube2,youtube3,snapchat1,snapchat2,snapchat3,tiktok1,tiktok2,tiktok3,raceid,racework,whitejob,race_sub1,race_sub2,voterid1,voterid2,serious,photo1,photo2,photo3,photo4,reinterview,birthyr,gender,educ,marstat,child18,race,employ,employ_t,faminc_new,votereg,ideo5,pid7,newsint,presvote16post,presvote16post_t,pew_bornagain,pew_religimp,pew_churatd,pew_prayer,religpew,religpew_t,religpew_protestant,religpew_protestant_t,inputstate,zipCode,FIPCounty,region,EnrollmentDate,CompletedSurveys,qualityControl_overall_scale,tsmart_P2012,tsmart_G2012,tsmart_P2016,tsmart_G2016,tsmart_P2018,tsmart_G2018,abortion1_skp,abortion2_skp,apppres5_skp,apppres7_skp,att1_skp,att2_skp,att3_skp,attother_skp,autism1_skp,autism2_skp,billtax_skp,buyback_skp,cexp1_grid_skp,cexp2_grid_skp,cheat_skp,combat_skp,confecon_skp,conspire1_skp,conspire2_skp,conspire3_skp,contact1a_skp,contact1b_skp,contact2a_skp,contact2b_skp,contact3_skp,corrupt_skp,cvote2020_skp,dem_activduty_skp,demo4_skp,disable_grid_skp,diversity5_skp,diversity7_skp,dreamer_skp,econnow_skp,econpres5_skp,econpres7_skp,electable_skp,elite1_skp,elite2_skp,elite3_skp,elite4_skp,excessive_skp,exercise_skp,exp1_grid_skp,exp2_grid_skp,experts_skp,exphelp_skp,facebook1_skp,facebook2_skp,facebook3_skp,finworry_skp,follow_skp,freecol_skp,frnpres5_skp,frnpres7_skp,ftasian_skp,ftb

To see a random selection of rows, use the `.sample(p, replace=False)` method, where `p` is the sample size and `replace=False` indicates sampling without replacement. To take a sample with replacement (for bootstrapping, for example), write `replace=True`. Here are five randomly drawn rows from `anes`:

In [8]:
anes.sample(5, replace=False)

,version,caseid,weight,weight_spss,form,follow,reg1a,reg1b,liveurban,youthurban,placeid1a,placeid1a_t,placeid1b,placeidimport,turnout16a,turnout16a1,turnout16b,turnout16c,vote16,turnout18a,turnout18a1,particip_1,particip_2,particip_3,particip_4,particip_5,particip_6,particip_7,particip_8,particip_9,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,vote20dem,vote20cand,vote20cand2,electable,vote20jb,vote20ew,vote20bs,cvote2020,tsplit1,contact1a,contact1b,contact2a,contact2b,contact3,contact4m_1,contact4m_2,contact4m_3,apppres5,frnpres5,immpres5,econpres5,apppres7,frnpres7,immpres7,econpres7,mip,econnow,finworry,confecon,improve1,national1,national2,conspire1,conspire2,conspire3,taxecon,billtax,trade1,trade2,trade3,trade4,richpoor2,guarinc,lcself,lcd,lcr,pop1,pop2,pop3,corrupt,immignum,refugees,dreamstr,dreamer,dreamstr1,dreamstr2,wall5,wall7,pathway,preturn,popen,release1,release2,famsep,tchina,trussia,tiran,tmexico,tjapan,hlthcare1,hlthcare2,abortion1,abortion2,freecol,loans,diversity5,diversity7,language,buyback,gw1,gw2,knowopioid1,knowopioid2,opioiddo,sentence,prek,demo4,experts,science,exphelp,elite1,elite2,elite3,elite4,ukraine1,ukraine2,excessive,rural1alt1,rural1alt2,rural2alt1,rural2alt2,rural3alt1,rural3alt2,rural4alt1,rural4alt2,conf_unemp,unemp,conf_interfere,interfere,conf_autism,autism1,autism2,conf_gmo,gmo1,gmo2,conf_warm,warm,conf_illegal,illegal,impeach1,impeach2,pk_cjus,pk_cjus_correct,pk_germ,pk_germ_correct,pk_sen,pk_spend,pk_geer,cheat,pid7x,pid1d,pid2d,pid1r,pid2r,pidstr,pidlean,ngun,shooting,dem_activduty,milyears,milyr1,milyr2,milyr3,milyr4,milyr5,milyr6,combat,harass1a,harass1b,rr1,rr2,rr3,rr4,health1a,health1b,hospital,feet,inches,nweight,bmi,disable1,disable2,disable3,disable4,disable5,disable6,smoker1,smoker2,exercise,relig1a,relig1a_txt,relig2a,att1,att2,att3,relig1b,relig1b_t,relig2b,relig3b,relig3b_t,attother,exptravel_ever,exphomesch,expfarm,expffood,expconvert,expholiday,explie,expshark,expdivorce,exparrest,expoverdose,expdefault,exppubasst,exphybrid,expmistake,explightning,exptravel_year,expindian,exphunt,expflag,exppublib,explottery,expshoponline,exppubtrans,expfight,expavoid,expknowimmig,expknowtrans,expbuyusa,expretire,expcolldebt,expknowpris,socmed_1,socmed_2,socmed_3,socmed_4,socmed_5,socmed_6,socmed_7,socmed_8,socmed_9,socmed_t,facebook1,facebook2,facebook3,twitter1,twitter2,twitter3,instagram1,instagram2,instagram3,reddit1,reddit2,reddit3,youtube1,youtube2,youtube3,snapchat1,snapchat2,snapchat3,tiktok1,tiktok2,tiktok3,raceid,racework,whitejob,race_sub1,race_sub2,voterid1,voterid2,serious,photo1,photo2,photo3,photo4,reinterview,birthyr,gender,educ,marstat,child18,race,employ,employ_t,faminc_new,votereg,ideo5,pid7,newsint,presvote16post,presvote16post_t,pew_bornagain,pew_religimp,pew_churatd,pew_prayer,religpew,religpew_t,religpew_protestant,religpew_protestant_t,inputstate,zipCode,FIPCounty,region,EnrollmentDate,CompletedSurveys,qualityControl_overall_scale,tsmart_P2012,tsmart_G2012,tsmart_P2016,tsmart_G2016,tsmart_P2018,tsmart_G2018,abortion1_skp,abortion2_skp,apppres5_skp,apppres7_skp,att1_skp,att2_skp,att3_skp,attother_skp,autism1_skp,autism2_skp,billtax_skp,buyback_skp,cexp1_grid_skp,cexp2_grid_skp,cheat_skp,combat_skp,confecon_skp,conspire1_skp,conspire2_skp,conspire3_skp,contact1a_skp,contact1b_skp,contact2a_skp,contact2b_skp,contact3_skp,corrupt_skp,cvote2020_skp,dem_activduty_skp,demo4_skp,disable_grid_skp,diversity5_skp,diversity7_skp,dreamer_skp,econnow_skp,econpres5_skp,econpres7_skp,electable_skp,elite1_skp,elite2_skp,elite3_skp,elite4_skp,excessive_skp,exercise_skp,exp1_grid_skp,exp2_grid_skp,experts_skp,exphelp_skp,facebook1_skp,facebook2_skp,facebook3_skp,finworry_skp,follow_skp,freecol_skp,frnpres5_skp,frnpres7_skp,ftasian_skp,ftb

To see the count of non-missing observations and summary statistics such as the mean, standard deviation, minimum, maximum, and quantiles for the numeric columns, use the `.describe()` method:

In [9]:
anes.describe()

,caseid,form,follow,reg1a,reg1b,liveurban,youthurban,placeid1a,placeid1b,placeidimport,turnout16a,turnout16a1,turnout16b,turnout16c,vote16,turnout18a,turnout18a1,particip_1,particip_2,particip_3,particip_4,particip_5,particip_6,particip_7,particip_8,particip_9,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,vote20dem,vote20cand,vote20cand2,electable,vote20jb,vote20ew,vote20bs,cvote2020,tsplit1,contact1a,contact1b,contact2a,contact2b,contact3,contact4m_1,contact4m_2,contact4m_3,apppres5,frnpres5,immpres5,econpres5,apppres7,frnpres7,immpres7,econpres7,econnow,finworry,confecon,improve1,national1,national2,conspire1,conspire2,conspire3,taxecon,billtax,trade1,trade2,trade3,trade4,richpoor2,guarinc,lcself,lcd,lcr,pop1,pop2,pop3,corrupt,immignum,refugees,dreamstr,dreamer,dreamstr1,dreamstr2,wall5,wall7,pathway,preturn,popen,release1,release2,famsep,tchina,trussia,tiran,tmexico,tjapan,hlthcare1,hlthcare2,abortion1,abortion2,freecol,loans,diversity5,diversity7,language,buyback,gw1,gw2,knowopioid1,knowopioid2,opioiddo,sentence,prek,demo4,experts,science,exphelp,elite1,elite2,elite3,elite4,ukraine1,ukraine2,excessive,rural1alt1,rural1alt2,rural2alt1,rural2alt2,rural3alt1,rural3alt2,rural4alt1,rural4alt2,conf_unemp,unemp,conf_interfere,interfere,conf_autism,autism1,autism2,conf_gmo,gmo1,gmo2,conf_warm,warm,conf_illegal,illegal,impeach1,impeach2,pk_cjus_correct,pk_germ_correct,pk_sen,pk_spend,pk_geer,cheat,pid7x,pid1d,pid1r,pidstr,pidlean,ngun,shooting,dem_activduty,milyears,milyr1,milyr2,milyr3,milyr4,milyr5,milyr6,combat,harass1a,harass1b,rr1,rr2,rr3,rr4,health1a,health1b,hospital,feet,inches,nweight,disable1,disable2,disable3,disable4,disable5,disable6,smoker1,smoker2,exercise,relig1a,relig2a,att1,att2,att3,relig1b,relig2b,relig3b,attother,exptravel_ever,exphomesch,expfarm,expffood,expconvert,expholiday,explie,expshark,expdivorce,exparrest,expoverdose,expdefault,exppubasst,exphybrid,expmistake,explightning,exptravel_year,expindian,exphunt,expflag,exppublib,explottery,expshoponline,exppubtrans,expfight,expavoid,expknowimmig,expknowtrans,expbuyusa,expretire,expcolldebt,expknowpris,socmed_1,socmed_2,socmed_3,socmed_4,socmed_5,socmed_6,socmed_7,socmed_8,socmed_9,facebook1,facebook2,facebook3,twitter1,twitter2,twitter3,instagram1,instagram2,instagram3,reddit1,reddit2,reddit3,youtube1,youtube2,youtube3,snapchat1,snapchat2,snapchat3,tiktok1,tiktok2,tiktok3,raceid,racework,whitejob,race_sub1,race_sub2,voterid1,voterid2,serious,photo1,photo2,photo3,photo4,reinterview,birthyr,gender,educ,marstat,child18,race,employ,faminc_new,votereg,ideo5,pid7,newsint,presvote16post,pew_bornagain,pew_religimp,pew_churatd,pew_prayer,religpew,religpew_protestant,inputstate,zipCode,region,qualityControl_overall_scale,abortion1_skp,abortion2_skp,apppres5_skp,apppres7_skp,att1_skp,att2_skp,att3_skp,attother_skp,autism1_skp,autism2_skp,billtax_skp,buyback_skp,cexp1_grid_skp,cexp2_grid_skp,cheat_skp,combat_skp,confecon_skp,conspire1_skp,conspire2_skp,conspire3_skp,contact1a_skp,contact1b_skp,contact2a_skp,contact2b_skp,contact3_skp,corrupt_skp,cvote2020_skp,dem_activduty_skp,demo4_skp,disable_grid_skp,diversity5_skp,diversity7_skp,dreamer_skp,econnow_skp,econpres5_skp,econpres7_skp,electable_skp,elite1_skp,elite2_skp,elite3_skp,elite4_skp,excessive_skp,exercise_skp,exp1_grid_skp,exp2_grid_skp,experts_skp,exphelp_skp,facebook1_skp,facebook2_skp,facebook3_skp,finworry_skp,follow_skp,freecol_skp,frnpres5_skp,frnpres7_skp,ftasian_skp,ftbiden_skp,ftblack_skp,ftbritain_skp,ftbuttigieg_skp,ftcanada_skp,ftchina_skp,ftfrance_skp,ftgermany_skp,ftharris_skp,fthisp_skp,ftice_skp,ftillegal_skp,ftimmig1_skp,ftimmig2_skp,ftiran_skp,ftisrael_skp,ftjapan_skp,ftjournal_skp,ftmexico_skp,ftmuslim_skp,ftnato_skp,ftnkorea_skp,ftnra_skp,ftobama_

To change the quantiles that are displayed, use the `percentiles` argument, set equal to a list of the desired quantiles. To see the 33rd and 67th quantiles, type

In [10]:
anes.describe(percentiles = [.33, .67])

,caseid,form,follow,reg1a,reg1b,liveurban,youthurban,placeid1a,placeid1b,placeidimport,turnout16a,turnout16a1,turnout16b,turnout16c,vote16,turnout18a,turnout18a1,particip_1,particip_2,particip_3,particip_4,particip_5,particip_6,particip_7,particip_8,particip_9,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,vote20dem,vote20cand,vote20cand2,electable,vote20jb,vote20ew,vote20bs,cvote2020,tsplit1,contact1a,contact1b,contact2a,contact2b,contact3,contact4m_1,contact4m_2,contact4m_3,apppres5,frnpres5,immpres5,econpres5,apppres7,frnpres7,immpres7,econpres7,econnow,finworry,confecon,improve1,national1,national2,conspire1,conspire2,conspire3,taxecon,billtax,trade1,trade2,trade3,trade4,richpoor2,guarinc,lcself,lcd,lcr,pop1,pop2,pop3,corrupt,immignum,refugees,dreamstr,dreamer,dreamstr1,dreamstr2,wall5,wall7,pathway,preturn,popen,release1,release2,famsep,tchina,trussia,tiran,tmexico,tjapan,hlthcare1,hlthcare2,abortion1,abortion2,freecol,loans,diversity5,diversity7,language,buyback,gw1,gw2,knowopioid1,knowopioid2,opioiddo,sentence,prek,demo4,experts,science,exphelp,elite1,elite2,elite3,elite4,ukraine1,ukraine2,excessive,rural1alt1,rural1alt2,rural2alt1,rural2alt2,rural3alt1,rural3alt2,rural4alt1,rural4alt2,conf_unemp,unemp,conf_interfere,interfere,conf_autism,autism1,autism2,conf_gmo,gmo1,gmo2,conf_warm,warm,conf_illegal,illegal,impeach1,impeach2,pk_cjus_correct,pk_germ_correct,pk_sen,pk_spend,pk_geer,cheat,pid7x,pid1d,pid1r,pidstr,pidlean,ngun,shooting,dem_activduty,milyears,milyr1,milyr2,milyr3,milyr4,milyr5,milyr6,combat,harass1a,harass1b,rr1,rr2,rr3,rr4,health1a,health1b,hospital,feet,inches,nweight,disable1,disable2,disable3,disable4,disable5,disable6,smoker1,smoker2,exercise,relig1a,relig2a,att1,att2,att3,relig1b,relig2b,relig3b,attother,exptravel_ever,exphomesch,expfarm,expffood,expconvert,expholiday,explie,expshark,expdivorce,exparrest,expoverdose,expdefault,exppubasst,exphybrid,expmistake,explightning,exptravel_year,expindian,exphunt,expflag,exppublib,explottery,expshoponline,exppubtrans,expfight,expavoid,expknowimmig,expknowtrans,expbuyusa,expretire,expcolldebt,expknowpris,socmed_1,socmed_2,socmed_3,socmed_4,socmed_5,socmed_6,socmed_7,socmed_8,socmed_9,facebook1,facebook2,facebook3,twitter1,twitter2,twitter3,instagram1,instagram2,instagram3,reddit1,reddit2,reddit3,youtube1,youtube2,youtube3,snapchat1,snapchat2,snapchat3,tiktok1,tiktok2,tiktok3,raceid,racework,whitejob,race_sub1,race_sub2,voterid1,voterid2,serious,photo1,photo2,photo3,photo4,reinterview,birthyr,gender,educ,marstat,child18,race,employ,faminc_new,votereg,ideo5,pid7,newsint,presvote16post,pew_bornagain,pew_religimp,pew_churatd,pew_prayer,religpew,religpew_protestant,inputstate,zipCode,region,qualityControl_overall_scale,abortion1_skp,abortion2_skp,apppres5_skp,apppres7_skp,att1_skp,att2_skp,att3_skp,attother_skp,autism1_skp,autism2_skp,billtax_skp,buyback_skp,cexp1_grid_skp,cexp2_grid_skp,cheat_skp,combat_skp,confecon_skp,conspire1_skp,conspire2_skp,conspire3_skp,contact1a_skp,contact1b_skp,contact2a_skp,contact2b_skp,contact3_skp,corrupt_skp,cvote2020_skp,dem_activduty_skp,demo4_skp,disable_grid_skp,diversity5_skp,diversity7_skp,dreamer_skp,econnow_skp,econpres5_skp,econpres7_skp,electable_skp,elite1_skp,elite2_skp,elite3_skp,elite4_skp,excessive_skp,exercise_skp,exp1_grid_skp,exp2_grid_skp,experts_skp,exphelp_skp,facebook1_skp,facebook2_skp,facebook3_skp,finworry_skp,follow_skp,freecol_skp,frnpres5_skp,frnpres7_skp,ftasian_skp,ftbiden_skp,ftblack_skp,ftbritain_skp,ftbuttigieg_skp,ftcanada_skp,ftchina_skp,ftfrance_skp,ftgermany_skp,ftharris_skp,fthisp_skp,ftice_skp,ftillegal_skp,ftimmig1_skp,ftimmig2_skp,ftiran_skp,ftisrael_skp,ftjapan_skp,ftjournal_skp,ftmexico_skp,ftmuslim_skp,ftnato_skp,ftnkorea_skp,ftnra_skp,ftobama_

To see the count of non-missing observations, number of unique values, and the most frequent value and its frequency for string or categorical features, use the `include='object'` argument:

In [11]:
anes.describe(include='object')

,version,weight,weight_spss,placeid1a_t,mip,pk_cjus,pk_germ,pid2d,pid2r,bmi,relig1a_txt,relig1b_t,relig3b_t,socmed_t,employ_t,presvote16post_t,religpew_t,religpew_protestant_t,FIPCounty,EnrollmentDate,CompletedSurveys,tsmart_P2012,tsmart_G2012,tsmart_P2016,tsmart_G2016,tsmart_P2018,tsmart_G2018,ord_ftillegal,ord_electable_1,ord_electable_2,ord_tchina,ord_trussia,ord_tiran,ord_tmexico,ord_tjapan,ord_disable1,ord_disable2,ord_disable3,ord_disable4,ord_disable5,ord_disable6,ord_exptravel_ever,ord_exphomesch,ord_expfarm,ord_expffood,ord_expconvert,ord_expholiday,ord_explie,ord_expshark,ord_expdivorce,ord_exparrest,ord_expoverdose,ord_expdefault,ord_exppubasst,ord_exphybrid,ord_expmistake,ord_explightning,ord_exptravel_year,ord_expindian,ord_exphunt,ord_expflag,ord_exppublib,ord_explottery,ord_expshoponline,ord_exppubtrans,ord_expfight,ord_expavoid,ord_expknowimmig,ord_expknowtrans,ord_expbuyusa,ord_expretire,ord_expcolldebt,ord_expknowpris,starttime,endtime,pop_density_public,flag_state
count,3165,3165,3165,3165,3158,3143,3143,3165,3164,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165,3165
unique,1,825,825,43,2073,511,599,22,12,1381,1,1,1,49,4,1,4,1,1,2027,818,2,2,2,2,2,2,1,3,3,6,6,6,6,6,7,7,7,7,7,7,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,6,6,6,6,6,6,6,6,6,6,4,4,4,4,4,4,3132,3146,259,4
top,ANES 2019 Pilot Study version 20200204,,,__NA__,Donald Trump,chief justice,chancellor of germany,__NA__,__NA__,,"-3, restricted access","-3, restricted access","-3, restricted access",__NA__,__NA__,__NA__,__NA__,"-3, restricted access","-3, restricted acces",250,108,,1,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,12/31/2019 19:41:26,12/21/2019 22:50:57,,0
freq,3165,165,165,3121,77,313,473,3140,3148,42,3165,3165,3165,3085,3162,3165,3159,3165,3165,13,17,2394,1633,2166,1773,2164,1587,3165,1769,1769,1579,1579,1579,1579,1579,1586,1586,1586,1586,1586,1586,1579,1579,1579,1579,1579,1579,1579,1579,1586,1586,1586,1586,1586,1586,1586,1586,1579,1579,1579,1579,1579,1586,1586,1586,1586,1586,1579,1579,1579,1586,1586,1586,2,2,188,2901


To see the name and data type of each column, use the `.dtypes` attribute. To avoid truncation, type `pd.set_option('display.max_rows', None)`. Because there are 900 columns to report in this case, to save space I won't turn off the truncation:

In [12]:
anes.dtypes

version               object
caseid                 int64
weight                object
weight_spss           object
form                   int64
                       ...  
starttime             object
endtime               object
duration               int64
pop_density_public    object
flag_state            object
Length: 900, dtype: object

To find rows in the data that are duplicated, use the `.duplicated()` method. This method outputs a list of values that are `True` or `False` indicating whether or not each row is a duplicate of another row in the dataframe. To view the duplicated rows, use the `.duplicated()` output to filter the rows when using `.loc` to subset the data (more on `.loc` below). By default, `.duplicated()` marks all duplicate rows as `True` except for the first occurrence in the dataframe. In the case of the ANES data, there are no duplicated rows:

In [13]:
anes.loc[anes.duplicated(),]

,version,caseid,weight,weight_spss,form,follow,reg1a,reg1b,liveurban,youthurban,placeid1a,placeid1a_t,placeid1b,placeidimport,turnout16a,turnout16a1,turnout16b,turnout16c,vote16,turnout18a,turnout18a1,particip_1,particip_2,particip_3,particip_4,particip_5,particip_6,particip_7,particip_8,particip_9,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,vote20dem,vote20cand,vote20cand2,electable,vote20jb,vote20ew,vote20bs,cvote2020,tsplit1,contact1a,contact1b,contact2a,contact2b,contact3,contact4m_1,contact4m_2,contact4m_3,apppres5,frnpres5,immpres5,econpres5,apppres7,frnpres7,immpres7,econpres7,mip,econnow,finworry,confecon,improve1,national1,national2,conspire1,conspire2,conspire3,taxecon,billtax,trade1,trade2,trade3,trade4,richpoor2,guarinc,lcself,lcd,lcr,pop1,pop2,pop3,corrupt,immignum,refugees,dreamstr,dreamer,dreamstr1,dreamstr2,wall5,wall7,pathway,preturn,popen,release1,release2,famsep,tchina,trussia,tiran,tmexico,tjapan,hlthcare1,hlthcare2,abortion1,abortion2,freecol,loans,diversity5,diversity7,language,buyback,gw1,gw2,knowopioid1,knowopioid2,opioiddo,sentence,prek,demo4,experts,science,exphelp,elite1,elite2,elite3,elite4,ukraine1,ukraine2,excessive,rural1alt1,rural1alt2,rural2alt1,rural2alt2,rural3alt1,rural3alt2,rural4alt1,rural4alt2,conf_unemp,unemp,conf_interfere,interfere,conf_autism,autism1,autism2,conf_gmo,gmo1,gmo2,conf_warm,warm,conf_illegal,illegal,impeach1,impeach2,pk_cjus,pk_cjus_correct,pk_germ,pk_germ_correct,pk_sen,pk_spend,pk_geer,cheat,pid7x,pid1d,pid2d,pid1r,pid2r,pidstr,pidlean,ngun,shooting,dem_activduty,milyears,milyr1,milyr2,milyr3,milyr4,milyr5,milyr6,combat,harass1a,harass1b,rr1,rr2,rr3,rr4,health1a,health1b,hospital,feet,inches,nweight,bmi,disable1,disable2,disable3,disable4,disable5,disable6,smoker1,smoker2,exercise,relig1a,relig1a_txt,relig2a,att1,att2,att3,relig1b,relig1b_t,relig2b,relig3b,relig3b_t,attother,exptravel_ever,exphomesch,expfarm,expffood,expconvert,expholiday,explie,expshark,expdivorce,exparrest,expoverdose,expdefault,exppubasst,exphybrid,expmistake,explightning,exptravel_year,expindian,exphunt,expflag,exppublib,explottery,expshoponline,exppubtrans,expfight,expavoid,expknowimmig,expknowtrans,expbuyusa,expretire,expcolldebt,expknowpris,socmed_1,socmed_2,socmed_3,socmed_4,socmed_5,socmed_6,socmed_7,socmed_8,socmed_9,socmed_t,facebook1,facebook2,facebook3,twitter1,twitter2,twitter3,instagram1,instagram2,instagram3,reddit1,reddit2,reddit3,youtube1,youtube2,youtube3,snapchat1,snapchat2,snapchat3,tiktok1,tiktok2,tiktok3,raceid,racework,whitejob,race_sub1,race_sub2,voterid1,voterid2,serious,photo1,photo2,photo3,photo4,reinterview,birthyr,gender,educ,marstat,child18,race,employ,employ_t,faminc_new,votereg,ideo5,pid7,newsint,presvote16post,presvote16post_t,pew_bornagain,pew_religimp,pew_churatd,pew_prayer,religpew,religpew_t,religpew_protestant,religpew_protestant_t,inputstate,zipCode,FIPCounty,region,EnrollmentDate,CompletedSurveys,qualityControl_overall_scale,tsmart_P2012,tsmart_G2012,tsmart_P2016,tsmart_G2016,tsmart_P2018,tsmart_G2018,abortion1_skp,abortion2_skp,apppres5_skp,apppres7_skp,att1_skp,att2_skp,att3_skp,attother_skp,autism1_skp,autism2_skp,billtax_skp,buyback_skp,cexp1_grid_skp,cexp2_grid_skp,cheat_skp,combat_skp,confecon_skp,conspire1_skp,conspire2_skp,conspire3_skp,contact1a_skp,contact1b_skp,contact2a_skp,contact2b_skp,contact3_skp,corrupt_skp,cvote2020_skp,dem_activduty_skp,demo4_skp,disable_grid_skp,diversity5_skp,diversity7_skp,dreamer_skp,econnow_skp,econpres5_skp,econpres7_skp,electable_skp,elite1_skp,elite2_skp,elite3_skp,elite4_skp,excessive_skp,exercise_skp,exp1_grid_skp,exp2_grid_skp,experts_skp,exphelp_skp,facebook1_skp,facebook2_skp,facebook3_skp,finworry_skp,follow_skp,freecol_skp,frnpres5_skp,frnpres7_skp,ftasian_skp,ftb

To generate a dataframe without these duplicted rows, use the `~` operator to negate the logical values. That tells Python to keep the rows that are not duplicated:

In [14]:
anes.loc[~anes.duplicated(),]

,version,caseid,weight,weight_spss,form,follow,reg1a,reg1b,liveurban,youthurban,placeid1a,placeid1a_t,placeid1b,placeidimport,turnout16a,turnout16a1,turnout16b,turnout16c,vote16,turnout18a,turnout18a1,particip_1,particip_2,particip_3,particip_4,particip_5,particip_6,particip_7,particip_8,particip_9,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,vote20dem,vote20cand,vote20cand2,electable,vote20jb,vote20ew,vote20bs,cvote2020,tsplit1,contact1a,contact1b,contact2a,contact2b,contact3,contact4m_1,contact4m_2,contact4m_3,apppres5,frnpres5,immpres5,econpres5,apppres7,frnpres7,immpres7,econpres7,mip,econnow,finworry,confecon,improve1,national1,national2,conspire1,conspire2,conspire3,taxecon,billtax,trade1,trade2,trade3,trade4,richpoor2,guarinc,lcself,lcd,lcr,pop1,pop2,pop3,corrupt,immignum,refugees,dreamstr,dreamer,dreamstr1,dreamstr2,wall5,wall7,pathway,preturn,popen,release1,release2,famsep,tchina,trussia,tiran,tmexico,tjapan,hlthcare1,hlthcare2,abortion1,abortion2,freecol,loans,diversity5,diversity7,language,buyback,gw1,gw2,knowopioid1,knowopioid2,opioiddo,sentence,prek,demo4,experts,science,exphelp,elite1,elite2,elite3,elite4,ukraine1,ukraine2,excessive,rural1alt1,rural1alt2,rural2alt1,rural2alt2,rural3alt1,rural3alt2,rural4alt1,rural4alt2,conf_unemp,unemp,conf_interfere,interfere,conf_autism,autism1,autism2,conf_gmo,gmo1,gmo2,conf_warm,warm,conf_illegal,illegal,impeach1,impeach2,pk_cjus,pk_cjus_correct,pk_germ,pk_germ_correct,pk_sen,pk_spend,pk_geer,cheat,pid7x,pid1d,pid2d,pid1r,pid2r,pidstr,pidlean,ngun,shooting,dem_activduty,milyears,milyr1,milyr2,milyr3,milyr4,milyr5,milyr6,combat,harass1a,harass1b,rr1,rr2,rr3,rr4,health1a,health1b,hospital,feet,inches,nweight,bmi,disable1,disable2,disable3,disable4,disable5,disable6,smoker1,smoker2,exercise,relig1a,relig1a_txt,relig2a,att1,att2,att3,relig1b,relig1b_t,relig2b,relig3b,relig3b_t,attother,exptravel_ever,exphomesch,expfarm,expffood,expconvert,expholiday,explie,expshark,expdivorce,exparrest,expoverdose,expdefault,exppubasst,exphybrid,expmistake,explightning,exptravel_year,expindian,exphunt,expflag,exppublib,explottery,expshoponline,exppubtrans,expfight,expavoid,expknowimmig,expknowtrans,expbuyusa,expretire,expcolldebt,expknowpris,socmed_1,socmed_2,socmed_3,socmed_4,socmed_5,socmed_6,socmed_7,socmed_8,socmed_9,socmed_t,facebook1,facebook2,facebook3,twitter1,twitter2,twitter3,instagram1,instagram2,instagram3,reddit1,reddit2,reddit3,youtube1,youtube2,youtube3,snapchat1,snapchat2,snapchat3,tiktok1,tiktok2,tiktok3,raceid,racework,whitejob,race_sub1,race_sub2,voterid1,voterid2,serious,photo1,photo2,photo3,photo4,reinterview,birthyr,gender,educ,marstat,child18,race,employ,employ_t,faminc_new,votereg,ideo5,pid7,newsint,presvote16post,presvote16post_t,pew_bornagain,pew_religimp,pew_churatd,pew_prayer,religpew,religpew_t,religpew_protestant,religpew_protestant_t,inputstate,zipCode,FIPCounty,region,EnrollmentDate,CompletedSurveys,qualityControl_overall_scale,tsmart_P2012,tsmart_G2012,tsmart_P2016,tsmart_G2016,tsmart_P2018,tsmart_G2018,abortion1_skp,abortion2_skp,apppres5_skp,apppres7_skp,att1_skp,att2_skp,att3_skp,attother_skp,autism1_skp,autism2_skp,billtax_skp,buyback_skp,cexp1_grid_skp,cexp2_grid_skp,cheat_skp,combat_skp,confecon_skp,conspire1_skp,conspire2_skp,conspire3_skp,contact1a_skp,contact1b_skp,contact2a_skp,contact2b_skp,contact3_skp,corrupt_skp,cvote2020_skp,dem_activduty_skp,demo4_skp,disable_grid_skp,diversity5_skp,diversity7_skp,dreamer_skp,econnow_skp,econpres5_skp,econpres7_skp,electable_skp,elite1_skp,elite2_skp,elite3_skp,elite4_skp,excessive_skp,exercise_skp,exp1_grid_skp,exp2_grid_skp,experts_skp,exphelp_skp,facebook1_skp,facebook2_skp,facebook3_skp,finworry_skp,follow_skp,freecol_skp,frnpres5_skp,frnpres7_skp,ftasian_skp,ftb

To examine the values of a specific column, call the column name either as an attribute of the dataframe (for example, to see people's "feeling thermometer" ratings of Joe Biden, type `anes.ftbiden`) or as a string within a list (`anes['ftbiden']`). Then it is possible to employ methods like `.describe()` on this single column:

In [15]:
anes['ftbiden'].describe() # or anes.ftbiden.describe()

count    3165.000000
mean       56.269194
std       120.196311
min        -7.000000
25%         7.000000
50%        44.000000
75%        71.000000
max       997.000000
Name: ftbiden, dtype: float64

For categorical columns, to list of the unique categories and their frequencies, use the `.value_counts()` method:

In [16]:
anes.vote20cand.value_counts()

-1    1769
 1     419
 7     281
 6     267
 3     133
 9      71
 4      71
 5      58
 8      57
 2      38
-7       1
Name: vote20cand, dtype: int64

There's a nicer-looking and more informative version of this frequency table available in the `sidetable` package, which adds the `.stb.freq()` method to a `pandas` dataframe. Pass a list with the column name to this function:

In [17]:
anes.stb.freq(['vote20cand'])

,vote20cand,Count,Percent,Cumulative Count,Cumulative Percent
0,-1,1769,0.558926,1769,0.558926
1,1,419,0.132385,2188,0.691311
2,7,281,0.088784,2469,0.780095
3,6,267,0.084360,2736,0.864455
4,3,133,0.042022,2869,0.906477
5,9,71,0.022433,2940,0.928910
6,4,71,0.022433,3011,0.951343
7,5,58,0.018325,3069,0.969668
8,8,57,0.018009,3126,0.987678
9,2,38,0.012006,3164,0.999684


## Indexing a Data Frame
Indexing a dataframe is the act of extracting a selection of rows or columns from a dataframe based on either the numeric position of the row or column, or based on the name of the row or column. There are four ways to index a dataframe:

* Using the `.iloc` attribute of the dataframe. This attribute allows us to extract certain rows and columns from a dataframe by directly entering in the row and column numbers of the selection.

* Using the `.loc` attribute of the dataframe. This attribute allows us to extract rows and columns based on the row and column names.

* Calling a specific column by name as an attribute of a dataframe.

* Using square brackets `[]` after writing the dataframe's name to extract one or more columns by referring to those columns' names. 

To extract based on numeric position, use the `.iloc` attribute of a dataframe as follows:
```
df.iloc[rownumbers, columnnumbers]
```
`.iloc` provides a version of the dataframe that can take row and column coordinates as the first and second elements within an associated list, like an array. To specify more than one row or more than one column, these elements can be lists or can use `:` to express a range. For example, to extract rows 1 through 4 (remembering that the first row is row 0 and tha the second number in the range is excluded) and columns 12 through 14, we can type:

In [18]:
anes.iloc[1:5, 12:15]

,placeid1b,placeidimport,turnout16a
1,-1,2,1
2,-1,4,1
3,1,2,-1
4,1,2,-1


To extract rows 2, 4, 6, and 8 for columns 20, 25, and 30, we type:

In [19]:
anes.iloc[[2,4,6,8], [20, 25, 30]]

,turnout18a1,particip_5,fttrump
2,-1,2,0
4,-1,2,94
6,-1,1,0
8,-1,1,61


The advantage of the `.loc` approach to extract rows and columns by name is the ability to use **slicing** - that is, specifying a range of columns from the leftmost to the rightmost column, and all columns in between. Writing `:` alone in the rows slot returns all of the rows. For example, the first of the feeling therometer features from left-to-right in the `anes` data is `fttrump` and the last column is `ftpales`. To extract all of these columns, we type:

In [20]:
anes.loc[:,'fttrump':'ftpales']

,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales
0,47,90,52,52,49,997,50,99,99,99,100,88,79,97,-1,99,82,71,86,88,90,66,89,88,81,77,98,94,89,88,99,99,92,89,86
1,41,30,41,17,31,30,29,91,96,92,93,93,25,94,-1,67,86,78,91,93,44,19,93,25,82,22,89,91,94,71,66,100,20,25,77
2,0,91,88,15,60,70,68,48,49,49,49,50,39,69,-1,63,66,51,40,2,2,2,1,3,59,1,50,1,1,1,51,87,50,1,3
3,100,50,0,0,0,0,0,0,0,0,0,0,0,-1,100,25,75,75,75,100,25,50,15,50,50,15,85,10,85,50,75,50,75,25,0
4,94,18,25,1,10,16,7,93,94,94,94,50,9,-1,92,20,68,16,94,84,23,29,50,51,49,5,73,44,72,97,58,96,60,45,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3160,81,20,7,4,6,1,2,94,94,94,95,21,31,-1,85,11,30,16,95,91,26,10,22,20,30,4,70,19,90,60,61,60,10,25,10
3161,35,41,25,61,92,3,1,45,56,53,69,11,3,-1,50,52,40,55,50,7,20,6,26,1,55,2,60,60,56,50,66,80,50,31,50
3162,6,31,50,2,59,0,31,88,100,100,100,60,100,-1,100,51,3,60,94,41,100,60,100,78,40,59,100,72,99,99,99,100,51,41,87
3163,1,100,95,62,79,59,51,100,65,50,52,50,51,-1,49,91,84,72,59,0,54,0,62,0,60,0,73,55,50,1,56,89,0,0,1


We can return one column by calling its name just like any other attribute by typing the dataframe, a period, and a column name. This approach is appropriate when we want to extract just one column for the datadrame. To extract the `ftbiden` column from `anes`, we type:

In [21]:
anes.ftbiden

0       52
1       41
2       88
3        0
4       25
        ..
3160     7
3161    25
3162    50
3163    95
3164    70
Name: ftbiden, Length: 3165, dtype: int64

Equivalently, we can call this column within brackets as follows:

In [22]:
anes['ftbiden']

0       52
1       41
2       88
3        0
4       25
        ..
3160     7
3161    25
3162    50
3163    95
3164    70
Name: ftbiden, Length: 3165, dtype: int64

This approach allows us to extract more than one column by including a list of column names in the index:

In [23]:
anes[['ftbiden', 'fttrump']]

,ftbiden,fttrump
0,52,47
1,41,41
2,88,0
3,0,100
4,25,94
...,...,...
3160,7,81
3161,25,35
3162,50,6
3163,95,1


## Selecting, Renaming, and Rearranging Columns
The `anes` data has 900 columns, which is too many for us to feasibly work with in this notebook. Even if the intention is to run a machine learning model that processes a large number of columns, it is probably the case that some columns should not be included in the feature space. The first task in data manipulation is to restrict the data to only those columns we actually need.

For the ANES, let's keep the following columns:

* `caseid` - the primary key that identifies individual survey respondents
* `liveurban` - Do you currently live in a rural area, small town, suburb, or a city?
* `vote16` - In the 2016 presidential election, who did you vote for?
* `particip_3` - In the last 12 months, have you joined in a protest march, rally, or demonstration?
* `vote20jb` - If the election is between Donald Trump and Joe Biden, who will you vote for?
* `mip` - What do you think is the most important problem facing this country? (free response)
* `confecon` - Overall, how worried are you about the national economy?
* `ideo5` - How would you describe your personal political ideology?
* `pid7` - Which party to you identify with?
* `guarinc` - Do you favor or oppose a universal basic income?
* `famsep` - Do you favor or oppose the policy of family separation at the border?
* `freecol` - Do you favor or oppose a policy of free tuition at public universities?
* `loans` - Do you favor or oppose a policy of student loan debt forgiveness?
* `race` - Race
* `birthyr` - Birth year
* `gender` - Gender
* `educ` - Educational attainment
* `inputstate` - State of residence
* `weight` - The survey probability weights that need to be applied to rows to make results more representative of the U.S. adult population

The easiest way to reduce the dataframe to include only these columns is to define a list of these column names, then pass the list to the dataframe index as follows:

In [24]:
mycols = ['caseid','liveurban','vote16','particip_3','vote20jb',
          'mip','confecon','ideo5','pid7','guarinc','famsep',
          'freecol','loans','race','birthyr','gender','educ','inputstate', 'weight']
anes[mycols]

,caseid,liveurban,vote16,particip_3,vote20jb,mip,confecon,ideo5,pid7,guarinc,famsep,freecol,loans,race,birthyr,gender,educ,inputstate,weight
0,1,3,3,2,2,Health Care,2,4,2,2,5,2,3,3,1969,1,2,48,1.34719693063187
1,2,3,1,2,1,Working together,2,4,6,6,4,5,6,1,1942,1,6,1,.780822076219216
2,3,1,2,2,2,health care,5,3,1,4,5,6,7,1,1954,2,2,37,.966366930694957
3,4,4,2,2,1,The economy.,2,3,1,4,3,3,2,2,1979,1,3,34,1.10348514780374
4,5,4,1,2,1,China,1,4,5,7,3,7,7,1,1940,2,6,48,1.09069730256741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3160,3161,2,1,2,1,The infiltration of Marxists into the institut...,2,4,5,7,2,7,7,1,1948,1,4,41,
3161,3162,4,3,2,3,Lack of basic resources being provided and off...,5,6,3,1,3,3,2,1,1996,1,2,45,7.03646496881757
3162,3163,4,-1,2,4,donald trump,4,2,4,7,5,7,7,3,1980,1,4,47,.892833236147303
3163,3164,3,-1,2,2,Donald Trump,3,2,2,2,4,4,5,2,1960,1,1,6,1.58161278448241


Note that while we displayed the reduced dataframe, we have not yet saved this reduced dataframe as a Python variable. 

In addition to the features listed above, let's also include the columns that contain the "feeling thermometers". The respondents are asked to rate many things from 0 (strongly dislike) to 100 (strongly like) including politicians like Joe Biden and Donald Trump, countries like Turkey and Canada, groups of people like Asians, immigrants, and journalists, and organizations like NATO and the NRA. These columns all begin with "ft": the ratings for Donald Trump and Joe Biden are contained in `fttrump` and `ftbiden`, for example. Other columns contain metadata regarding these responses: `ftbiden_skp` indicates whether the respondent skipped this question, `ord_ftbiden` reports where this question appeared in each respondent's randomized question ordering, and `ftbiden_page_timing` records how long the respondent took to answer this question. For this example, we only want to keep the feeling thermometers.

In this case, we want to use code that automatically identifies all of the columns that begin with a specific string like "ft" or end with a specific string like "timing". There are two approaches to identifying and selecting columns that start or end a certain way. First, we can use the `.str.startswith()`, `.str.endswith()`, and `str.contains()` methods of the `.columns` attribute of a dataframe, and pass the result to the `.loc` attribute to extract the matching columns. To extract the columns that begin "ft", we write

In [25]:
anes.loc[:, anes.columns.str.startswith('ft')]

,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,ftasian_skp,ftbiden_skp,ftblack_skp,ftbritain_skp,ftbuttigieg_skp,ftcanada_skp,ftchina_skp,ftfrance_skp,ftgermany_skp,ftharris_skp,fthisp_skp,ftice_skp,ftillegal_skp,ftimmig1_skp,ftimmig2_skp,ftiran_skp,ftisrael_skp,ftjapan_skp,ftjournal_skp,ftmexico_skp,ftmuslim_skp,ftnato_skp,ftnkorea_skp,ftnra_skp,ftobama_skp,ftpales_skp,ftrussia_skp,ftsanders_skp,ftsaudi_skp,fttrump_skp,ftturkey_skp,ftukraine_skp,ftun_skp,ftwarren_skp,ftwhite_skp,fttrump_page_timing,ftobama_page_timing,ftbiden_page_timing,ftwarren_page_timing,ftsanders_page_timing,ftbuttigieg_page_timing,ftharris_page_timing,ftblack_page_timing,ftwhite_page_timing,fthisp_page_timing,ftasian_page_timing,ftmuslim_page_timing,ftillegal_page_timing,ftimmig1_page_timing,ftimmig2_page_timing,ftjournal_page_timing,ftnato_page_timing,ftun_page_timing,ftice_page_timing,ftnra_page_timing,ftchina_page_timing,ftnkorea_page_timing,ftmexico_page_timing,ftsaudi_page_timing,ftukraine_page_timing,ftiran_page_timing,ftbritain_page_timing,ftgermany_page_timing,ftjapan_page_timing,ftisrael_page_timing,ftfrance_page_timing,ftcanada_page_timing,ftturkey_page_timing,ftrussia_page_timing,ftpales_page_timing
0,47,90,52,52,49,997,50,99,99,99,100,88,79,97,-1,99,82,71,86,88,90,66,89,88,81,77,98,94,89,88,99,99,92,89,86,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18.756,8.893,10.590,8.145,6.211,5.852,13.662,8.236,4.700,2.560,3.830,8.659,1.679,3.532,0.000,5.137,3.904,3.249,5.800,7.497,10.249,4.214,5.650,3.823,3.182,2.453,4.631,4.221,3.878,3.864,13.908,2.109,2.791,4.533,4.073
1,41,30,41,17,31,30,29,91,96,92,93,93,25,94,-1,67,86,78,91,93,44,19,93,25,82,22,89,91,94,71,66,100,20,25,77,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12.653,4.982,3.735,3.534,2.999,4.752,3.720,5.888,3.343,3.709,3.461,3.497,2.850,2.800,0.000,4.368,3.392,4.289,5.103,3.714,3.830,3.684,3.034,3.203,4.249,3.204,4.261,2.965,3.921,3.944,3.005,3.957,2.563,2.851,5.058
2,0,91,88,15,60,70,68,48,49,49,49,50,39,69,-1,63,66,51,40,2,2,2,1,3,59,1,50,1,1,1,51,87,50,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.342,8.833,5.336,6.344,9.386,7.735,7.817,4.727,4.530,6.723,4.873,6.401,11.542,7.602,0.000,5.969,6.955,10.404,20.956,9.733,4.223,5.717,5.417,6.487,9.220,3.874,13.394,4.204,8.374,4.453,5.695,4.635,4.992,6.135,5.758
3,100,50,0,0,0,0,0,0,0,0,0,0,0,-1,100,25,75,75,75,100,25,50,15,50,50,15,85,10,85,50,75,50,75,25,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6.123,4.024,7.260,4.035,8.174,4.913,2.623,2.955,5.917,3.436,5.724,5.769,7.385,0.000,8.174,17.756,3.771,0.898,15.432,4.706,8.266,5.902,0.105,8.623,0.298,0.344,8.744,8.606,0.306,0.233,0.118,3.571,0.185,0.139,0.137
4,94,18,25,1,10,16,7,93,94,94,94,50,9,-1,92,20,68,16,94,84,23,29,50,51,49,5,73,44,72,97,58,96,60,45,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23.543,16.482,12.024,7.980,13.030,15.878,5.829,8.313,27.213,9.984,7.684,20.659,10.477,0.000,7.723,6.285,11.818,7.484,9.420,13.706,8.573,9.659,6.841,9.907,8.938,7.456,6.822,9.949,10.211,7.729,11.479,16.550,25.597,10.016,8.424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3160,81,20,7,4,6,1,2,94,94,94,95,21,31,-1,85,11,30,16,95,91,26,10,22,20,30,4,70,19,90,60,61,60,10,25,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.782

The issue here is that this call included the page timing columns, which we did not want, because these columns also begin with "ft". In addition, this approach does not enable us to include the features listed above. 

A second approach to selecting columns is technically more complicated, but offers a great deal more control over which columns are included in the edited data and which columns are not. This approach involves using a comprehension loop on the `.columns` attribute to extract the elements of this list that match given patterns. The individual elements of `.columns` are strings on which `.startswith()`, `.endswith()`, and `.contains()` apply. The comprehension loop allows us to use these methods in logical statements that determine whether each element is included or excluded in the new list.

For example, 
```
[x for x in anes.colummns]
```
generates a list of all 900 column names in the `anes` dataframe. But we can use `if` to place conditions on which names are included in the list. If we type
```
[x for x in anes.colummns if x.startswith("ft")]
```
then names in `anes.columns` are included only if it is true that they start with "ft". However, that condition also captures the metadata columns we don't want that begin with "ft" and end with "timing" or "skp". We can exclude these columns by using `not` clauses in the logical condition as follows:

In [26]:
ftcols = [x for x in anes.columns if x.startswith("ft") and not x.endswith("timing") and not x.endswith("skp")]
ftcols

['fttrump',
 'ftobama',
 'ftbiden',
 'ftwarren',
 'ftsanders',
 'ftbuttigieg',
 'ftharris',
 'ftblack',
 'ftwhite',
 'fthisp',
 'ftasian',
 'ftmuslim',
 'ftillegal',
 'ftimmig1',
 'ftimmig2',
 'ftjournal',
 'ftnato',
 'ftun',
 'ftice',
 'ftnra',
 'ftchina',
 'ftnkorea',
 'ftmexico',
 'ftsaudi',
 'ftukraine',
 'ftiran',
 'ftbritain',
 'ftgermany',
 'ftjapan',
 'ftisrael',
 'ftfrance',
 'ftcanada',
 'ftturkey',
 'ftrussia',
 'ftpales']

To append two lists together, we simply add them together. Here we can append the `mycols` list defined above to the `ftcols` list to create a comprehensive list of all the columns we want to extract from `anes`, and we can create a new variable named `anes_clean` that contains only these columns:

In [27]:
anes_clean = anes[mycols + ftcols]
anes_clean

,caseid,liveurban,vote16,particip_3,vote20jb,mip,confecon,ideo5,pid7,guarinc,famsep,freecol,loans,race,birthyr,gender,educ,inputstate,weight,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales
0,1,3,3,2,2,Health Care,2,4,2,2,5,2,3,3,1969,1,2,48,1.34719693063187,47,90,52,52,49,997,50,99,99,99,100,88,79,97,-1,99,82,71,86,88,90,66,89,88,81,77,98,94,89,88,99,99,92,89,86
1,2,3,1,2,1,Working together,2,4,6,6,4,5,6,1,1942,1,6,1,.780822076219216,41,30,41,17,31,30,29,91,96,92,93,93,25,94,-1,67,86,78,91,93,44,19,93,25,82,22,89,91,94,71,66,100,20,25,77
2,3,1,2,2,2,health care,5,3,1,4,5,6,7,1,1954,2,2,37,.966366930694957,0,91,88,15,60,70,68,48,49,49,49,50,39,69,-1,63,66,51,40,2,2,2,1,3,59,1,50,1,1,1,51,87,50,1,3
3,4,4,2,2,1,The economy.,2,3,1,4,3,3,2,2,1979,1,3,34,1.10348514780374,100,50,0,0,0,0,0,0,0,0,0,0,0,-1,100,25,75,75,75,100,25,50,15,50,50,15,85,10,85,50,75,50,75,25,0
4,5,4,1,2,1,China,1,4,5,7,3,7,7,1,1940,2,6,48,1.09069730256741,94,18,25,1,10,16,7,93,94,94,94,50,9,-1,92,20,68,16,94,84,23,29,50,51,49,5,73,44,72,97,58,96,60,45,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3160,3161,2,1,2,1,The infiltration of Marxists into the institut...,2,4,5,7,2,7,7,1,1948,1,4,41,,81,20,7,4,6,1,2,94,94,94,95,21,31,-1,85,11,30,16,95,91,26,10,22,20,30,4,70,19,90,60,61,60,10,25,10
3161,3162,4,3,2,3,Lack of basic resources being provided and off...,5,6,3,1,3,3,2,1,1996,1,2,45,7.03646496881757,35,41,25,61,92,3,1,45,56,53,69,11,3,-1,50,52,40,55,50,7,20,6,26,1,55,2,60,60,56,50,66,80,50,31,50
3162,3163,4,-1,2,4,donald trump,4,2,4,7,5,7,7,3,1980,1,4,47,.892833236147303,6,31,50,2,59,0,31,88,100,100,100,60,100,-1,100,51,3,60,94,41,100,60,100,78,40,59,100,72,99,99,99,100,51,41,87
3163,3164,3,-1,2,2,Donald Trump,3,2,2,2,4,4,5,2,1960,1,1,6,1.58161278448241,1,100,95,62,79,59,51,100,65,50,52,50,51,-1,49,91,84,72,59,0,54,0,62,0,60,0,73,55,50,1,56,89,0,0,1


A related method is choosing which columns in a dataframe to drop, instead of which columns to keep. This technique is useful when we want to keep most of the columns, and it is easier to specify the few columns we want to drop. The method to drop columns is `.drop()`, applied to a dataframe. There are two ways to use this method. First,
```
df = df.drop(colstodrop, axis=1)
```
where `colstodrop` is a vector of column names for the columns we want to drop. Alternatively, we can write
```
df.drop(colstodrop, axis=1, inplace=True)
```
These two versions of the command have the same effect of dropping the columns defined in `colstodrop` and overwriting the Python variable for the dataframe with the new version. `axis=1` specifies that the names refer to columns to be dropped, in contrast to `axis=0` which would refer to rows. `inplace=True` provides an equivalent to `df = `. Either approach is fine, although I tend to avoid using `inplace=True` so that the code is more consistent. To delete the `inputstate` column from `anes_clean`, we type

In [28]:
anes_clean = anes_clean.drop('inputstate', axis=1)

If we wanted to drop multiple columns, we would write a list of the column names we want to drop in the first argument.

There are many ways to rename the columns in a dataframe, including assigning a list of new column names to the `.columns` attribute of a dataframe. I do not recommend that approach, however, because it requires an entry for every column and it requires these names to be written in exactly the same order as the existing columns, or else the data will be corrupted. Instead, use the `.rename()` method. This method does not require us to think about the left-to-right order of the columns, and it allows us to rename only a few columns without worrying about the ones we do not want to rename. To use this method, specify two parameters: first a dictionary that contains elements in the form of `'oldname':'newname'`, and second `axis='columns'` or `axis=1` to work with columns. 

To rename some of the columns in `anes_clean`, I use the following code:

In [29]:
anes_clean = anes_clean.rename({'particip_3':'protest',
                                'vote20jb':'vote',
                                'mip':'most_important_issue',
                                'ideo5':'ideology',
                                'pid7':'partyID',
                                'guarinc':'universal_income',
                                'famsep':'family_separation',
                                'freecol':'free_college',
                                'loans':'forgive_loans',
                                'gender':'sex',
                                'educ':'education'}, axis=1)
anes_clean

,caseid,liveurban,vote16,protest,vote,most_important_issue,confecon,ideology,partyID,universal_income,family_separation,free_college,forgive_loans,race,birthyr,sex,education,weight,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales
0,1,3,3,2,2,Health Care,2,4,2,2,5,2,3,3,1969,1,2,1.34719693063187,47,90,52,52,49,997,50,99,99,99,100,88,79,97,-1,99,82,71,86,88,90,66,89,88,81,77,98,94,89,88,99,99,92,89,86
1,2,3,1,2,1,Working together,2,4,6,6,4,5,6,1,1942,1,6,.780822076219216,41,30,41,17,31,30,29,91,96,92,93,93,25,94,-1,67,86,78,91,93,44,19,93,25,82,22,89,91,94,71,66,100,20,25,77
2,3,1,2,2,2,health care,5,3,1,4,5,6,7,1,1954,2,2,.966366930694957,0,91,88,15,60,70,68,48,49,49,49,50,39,69,-1,63,66,51,40,2,2,2,1,3,59,1,50,1,1,1,51,87,50,1,3
3,4,4,2,2,1,The economy.,2,3,1,4,3,3,2,2,1979,1,3,1.10348514780374,100,50,0,0,0,0,0,0,0,0,0,0,0,-1,100,25,75,75,75,100,25,50,15,50,50,15,85,10,85,50,75,50,75,25,0
4,5,4,1,2,1,China,1,4,5,7,3,7,7,1,1940,2,6,1.09069730256741,94,18,25,1,10,16,7,93,94,94,94,50,9,-1,92,20,68,16,94,84,23,29,50,51,49,5,73,44,72,97,58,96,60,45,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3160,3161,2,1,2,1,The infiltration of Marxists into the institut...,2,4,5,7,2,7,7,1,1948,1,4,,81,20,7,4,6,1,2,94,94,94,95,21,31,-1,85,11,30,16,95,91,26,10,22,20,30,4,70,19,90,60,61,60,10,25,10
3161,3162,4,3,2,3,Lack of basic resources being provided and off...,5,6,3,1,3,3,2,1,1996,1,2,7.03646496881757,35,41,25,61,92,3,1,45,56,53,69,11,3,-1,50,52,40,55,50,7,20,6,26,1,55,2,60,60,56,50,66,80,50,31,50
3162,3163,4,-1,2,4,donald trump,4,2,4,7,5,7,7,3,1980,1,4,.892833236147303,6,31,50,2,59,0,31,88,100,100,100,60,100,-1,100,51,3,60,94,41,100,60,100,78,40,59,100,72,99,99,99,100,51,41,87
3163,3164,3,-1,2,2,Donald Trump,3,2,2,2,4,4,5,2,1960,1,1,1.58161278448241,1,100,95,62,79,59,51,100,65,50,52,50,51,-1,49,91,84,72,59,0,54,0,62,0,60,0,73,55,50,1,56,89,0,0,1


## Working with Categorical Features
### Recoding a Single Categorical Column
The ANES uses numeric codes to represent both ordinal and nominal (unordered) categorical features. That cuts down on the memory size of the data file (storing `2` is smaller than storing `Trump`), but it also makes it so we can't really understand what the individual datapoints represent. For example, here's a frequency table of the values of `vote`:

In [30]:
anes_clean.vote.value_counts()

2    1288
1    1273
3     321
4     283
Name: vote, dtype: int64

According to the codebook, these values mean
* 1 - Donald Trump
* 2 - Joe Biden
* 3 - Someone else
* 4 - Probably will not vote

It would be much better to replace the numeric datapoints with these labels so that we can better understand the data we see. Carefully replacing the numbers with text will keep us from potentially confusing which number stands for which category. Also, if we want to construct any tables or graphs, it's much better to display the text labels than the numeric codes. Replacing the numeric codes with text for the categorical labels is a two step process:

1. Create a dictionary in which the keys are existing values we want to recode, and the values are the new labels we want to replace these categories with.

2. Use the `.map()` method on the column of interest to apply the mapping defined by the dictionary to the data.

An alternative method is `.replace()` as applied to the entire dataframe, but `.map()` tends to be [much faster](https://stackoverflow.com/questions/41985566/pandas-replace-dictionary-slowness) than `.replace()`. For example, to put the text labels onto the `vote` column, we can first define the following dictionary, then we can apply it to the data:

In [31]:
replace_map = {1:'Donald Trump', 
               2:'Joe Biden', 
               3:'Someone else', 
               4:'Probably will not vote'}
anes_clean.vote = anes_clean.vote.map(replace_map)
anes_clean.vote

0                    Joe Biden
1                 Donald Trump
2                    Joe Biden
3                 Donald Trump
4                 Donald Trump
                 ...          
3160              Donald Trump
3161              Someone else
3162    Probably will not vote
3163                 Joe Biden
3164                 Joe Biden
Name: vote, Length: 3165, dtype: object

One drawback of `.map()` compared to `.replace()` is that *all* of the categories must be replaced. If only some categories need to be recoded and a dictionary only lists the categories to replace, then `.map()` replaces all of the unlisted categories with missing values, while `.replace()` leaves these categories as they are.

This method can also be used to collapse categories into a smaller number of categories and can be used to replace numeric codes that represent missing values with codes recognized as missing by Python. To combine categories, we set existing values to the same label. To turn some categories to missing values, we set those categories equal to `np.nan`. 

Take `ideology`, for example. According to the codebook, the values of this feature mean
* -7 - No answer
* 1 - Very liberal
* 2 - Liberal
* 3 - Moderate
* 4 - Conservative
* 5 - Very conservative
* 6 - Not sure
Suppose that we want to label these categories in a way that combines categories 1 and 2 to be "Liberal", sets category 3 as "Moderate", combines categories 4 and 5 to be "Conservative", and replaces categories -7 and 6 with missing values.  We can do so with the following dictionary:

In [32]:
replace_map = {-7:np.nan, 1:'Liberal', 2:'Liberal', 3:'Moderate', 
               4:'Conservative', 5:'Conservative', 6:np.nan}
anes_clean.ideology = anes_clean.ideology.map(replace_map)
anes_clean.ideology

0       Conservative
1       Conservative
2           Moderate
3           Moderate
4       Conservative
            ...     
3160    Conservative
3161             NaN
3162         Liberal
3163         Liberal
3164        Moderate
Name: ideology, Length: 3165, dtype: object

### Recoding Many Categorical Columns At Once
To label the categories of many features, the simplest code defines a dictionary in which each feature to be recoded is a key and the mapping dictionary for that feature is the value. Then we can pass this entire dictionary to the whole dataframe with `.replace()`. This approach might take longer than defining a loop that uses `.map()` on each of these columns (`.map()` is only defined for series and not for dataframes), but using `.replace()` here is more intuitive and elegant. 

We can recode all the remaining categorical features in the ANES data as follows:

In [33]:
replace_map = {'liveurban':{1:'Rural', 2:'Town', 3:'Suburb', 4:'City'},
              'vote16':{-1:'Did not vote', 1:'Donald Trump', 2:'Hillary Clinton', 3:'Someone else'},
              'protest':{1:True, 2:False},
              'confecon':{1:'Not at all worried', 2:'A little worried', 3:'Moderately worried', 4:'Very worried', 5:'Extremely worried'},
              'partyID':{-7:np.nan, 8: np.nan, 1:'Democrat', 2:'Democrat', 3:'Democrat', 4:'Independent', 5:'Republican', 
                         6:'Republican', 7:'Republican'},
              'universal_income':{1:'Favor a great deal', 2:'Favor a moderate amount', 3:'Favor a little', 4:'Neither favor nor oppose', 
                                  5:'Oppose a little', 6:'Oppose a moderate amount', 7:'Oppose a great deal'},
              'family_separation':{-7:np.nan, 1:'Favor strongly', 2:'Favor somewhat', 3:'Neither favor nor disagree',
                                   4:'Oppose somewhat', 5:'Oppose strongly'},
               'free_college':{-7:np.nan, 1:'Favor a great deal', 2:'Favor a moderate amount', 3:'Favor a little', 
                               4:'Neither favor nor oppose', 5:'Oppose a little', 6:'Oppose a moderate amount', 7:'Oppose a great deal'},
               'forgive_loans':{-7:np.nan, 1:'Favor a great deal', 2:'Favor a moderate amount', 3:'Favor a little', 
                               4:'Neither favor nor oppose', 5:'Oppose a little', 6:'Oppose a moderate amount', 7:'Oppose a great deal'},
               'race':{1:'White', 2:'Black', 3:'Hispanic', 4:'Other', 5:'Other', 6:'Other', 7:'Other', 8:'Other'},
               'sex':{1:'Male',2:'Female'},
               'education':{1:'No HS diploma', 2:'High school graduate', 3:'Some college', 4:'2-year degree', 
                            5:'4-year degree', 6:'Post-graduate'}}
anes_clean = anes_clean.replace(replace_map)

To apply the same recoding dictionary to many columns in the dataframe, first generate a list of the column names that share the dictionary. Then pass that list to the dataframe index to select those columns. Finally, apply the `.replace()` to this dataframe, inputting the dictionary for the shared recoding instructions. For example, for all of the feeling thermometer features in the data, various types of non-response are coded as -1, -7, and 997. I want to apply the dictionary
```
{-1:np.nan, -7:np.nan, 997:np.nan}
```
to every feeling thermometer feature in the data. If all of the values are to be recoded to the same single (missing) value, a shortcut is to pass to `.replace()` a list of the values to be recoded and `np.nan`. So we can equivalently write
```
.replace([-1, -7, 997], np.nan)
```
First we can generate a list of these columns, then we pass it to the dataframe. Then we apply `.replace()` to this selection: 

In [34]:
ftcols = [x for x in anes_clean.columns if x.startswith("ft")]
anes_clean[ftcols] = anes_clean[ftcols].replace([-1, -7, 997], np.nan)

All of the features in the data have now been recoded:

In [35]:
anes_clean

,caseid,liveurban,vote16,protest,vote,most_important_issue,confecon,ideology,partyID,universal_income,family_separation,free_college,forgive_loans,race,birthyr,sex,education,weight,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftimmig1,ftimmig2,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales
0,1,Suburb,Someone else,False,Joe Biden,Health Care,A little worried,Conservative,Democrat,Favor a moderate amount,Oppose strongly,Favor a moderate amount,Favor a little,Hispanic,1969,Male,High school graduate,1.34719693063187,47.0,90.0,52.0,52.0,49.0,NaN,50.0,99.0,99.0,99.0,100.0,88.0,79.0,97.0,NaN,99,82.0,71.0,86.0,88.0,90,66.0,89.0,88.0,81.0,77.0,98,94,89,88.0,99.0,99,92.0,89.0,86.0
1,2,Suburb,Donald Trump,False,Donald Trump,Working together,A little worried,Conservative,Republican,Oppose a moderate amount,Oppose somewhat,Oppose a little,Oppose a moderate amount,White,1942,Male,Post-graduate,.780822076219216,41.0,30.0,41.0,17.0,31.0,30.0,29.0,91.0,96.0,92.0,93.0,93.0,25.0,94.0,NaN,67,86.0,78.0,91.0,93.0,44,19.0,93.0,25.0,82.0,22.0,89,91,94,71.0,66.0,100,20.0,25.0,77.0
2,3,Rural,Hillary Clinton,False,Joe Biden,health care,Extremely worried,Moderate,Democrat,Neither favor nor oppose,Oppose strongly,Oppose a moderate amount,Oppose a great deal,White,1954,Female,High school graduate,.966366930694957,0.0,91.0,88.0,15.0,60.0,70.0,68.0,48.0,49.0,49.0,49.0,50.0,39.0,69.0,NaN,63,66.0,51.0,40.0,2.0,2,2.0,1.0,3.0,59.0,1.0,50,1,1,1.0,51.0,87,50.0,1.0,3.0
3,4,City,Hillary Clinton,False,Donald Trump,The economy.,A little worried,Moderate,Democrat,Neither favor nor oppose,Neither favor nor disagree,Favor a little,Favor a moderate amount,Black,1979,Male,Some college,1.10348514780374,100.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,100.0,25,75.0,75.0,75.0,100.0,25,50.0,15.0,50.0,50.0,15.0,85,10,85,50.0,75.0,50,75.0,25.0,0.0
4,5,City,Donald Trump,False,Donald Trump,China,Not at all worried,Conservative,Republican,Oppose a great deal,Neither favor nor disagree,Oppose a great deal,Oppose a great deal,White,1940,Female,Post-graduate,1.09069730256741,94.0,18.0,25.0,1.0,10.0,16.0,7.0,93.0,94.0,94.0,94.0,50.0,9.0,NaN,92.0,20,68.0,16.0,94.0,84.0,23,29.0,50.0,51.0,49.0,5.0,73,44,72,97.0,58.0,96,60.0,45.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3160,3161,Town,Donald Trump,False,Donald Trump,The infiltration of Marxists into the institut...,A little worried,Conservative,Republican,Oppose a great deal,Favor somewhat,Oppose a great deal,Oppose a great deal,White,1948,Male,2-year degree,,81.0,20.0,7.0,4.0,6.0,1.0,2.0,94.0,94.0,94.0,95.0,21.0,31.0,NaN,85.0,11,30.0,16.0,95.0,91.0,26,10.0,22.0,20.0,30.0,4.0,70,19,90,60.0,61.0,60,10.0,25.0,10.0
3161,3162,City,Someone else,False,Someone else,Lack of basic resources being provided and off...,Extremely worried,NaN,Democrat,Favor a great deal,Neither favor nor disagree,Favor a little,Favor a moderate amount,White,1996,Male,High school graduate,7.03646496881757,35.0,41.0,25.0,61.0,92.0,3.0,1.0,45.0,56.0,53.0,69.0,11.0,3.0,NaN,50.0,52,40.0,55.0,50.0,7.0,20,6.0,26.0,1.0,55.0,2.0,60,60,56,50.0,66.0,80,50.0,31.0,50.0
3162,3163,City,Did not vote,False,Probably will not vote,donald trump,Very worried,Liberal,Independent,Oppose a great deal,Oppose strongly,Oppose a great deal,Oppose a great deal,Hispanic,1980,Male,2-year degree,.892833236147303,6.0,31.0,50.0,2.0,59.0,0.0,31.0,88.0,100.0,100.0,100.0,60.0,100.0,NaN,100.0,51,3.0,60.0,94.0,41.0,100,60.0,100.0,78.0,40.0,59.0,100,72,99,99.0,99.0,100,51.0,41.0,87.0
3163,3164,Suburb,Did not vote,False,Joe Biden,Donald Trump,Moderately worried,Liberal,Democrat,Favor a moderate amount,Oppose somewhat,Neither favor nor oppo

### Dealing With Missing Data

After recoding, we are left with quite a few missing values in the data. To generate a logical matrix that indicates for every cell whether the datapoint is missing or not, we can use `anes_clean.isnull()`. To delete every row that has even one missing value, we can use `anes_clean.dropna()`. That's a heavy-handed treatment of missing data, however, that can severely reduce sample size and can lead to biases in analytical results of datapoints are missing not at random. There are much better treatments for missing data, called **imputation** methods. Many methods for missing data imputation are implemented in the [scikit-learn package](https://scikit-learn.org/stable/modules/impute.html).

One application of the `isnull()` method is to replace missing values with known values if those values are available elsewhere. For example, the ANES asked two versions of the feeling thermometer for immigrants. A respondent is only asked one of the two questions, so one of `ftimmig1` and `ftimmig2` is missing:

In [36]:
anes_clean[['ftimmig1','ftimmig2']]

,ftimmig1,ftimmig2
0,97.0,NaN
1,94.0,NaN
2,69.0,NaN
3,NaN,100.0
4,NaN,92.0
...,...,...
3160,NaN,85.0
3161,NaN,50.0
3162,NaN,100.0
3163,NaN,49.0


We can create a new column called `ftimmig` and set it equal to `ftimmig1`. Then we can subset this column to just the rows for which `ftimmig1` is missing by typing
```
anes_clean.ftimmig[anes_clean.ftimmig1.isnull()]
```
We can then set these values equal to `ftimmig2` for the same subset where `ftimmig1` is missing:

In [37]:
anes_clean['ftimmig'] = anes_clean.ftimmig1
anes_clean.ftimmig[anes_clean.ftimmig1.isnull()] = anes_clean.ftimmig2[anes_clean.ftimmig1.isnull()]
anes_clean[['ftimmig','ftimmig1','ftimmig2']]

,ftimmig,ftimmig1,ftimmig2
0,97.0,97.0,NaN
1,94.0,94.0,NaN
2,69.0,69.0,NaN
3,100.0,NaN,100.0
4,92.0,NaN,92.0
...,...,...,...
3160,85.0,NaN,85.0
3161,50.0,NaN,50.0
3162,100.0,NaN,100.0
3163,49.0,NaN,49.0


Now that we have created `ftimmig`, we can drop `ftimmig1` and `ftimmig2`:

In [38]:
anes_clean = anes_clean.drop(['ftimmig1','ftimmig2'], axis=1)

### Why Recoding Categorical Data is Dangerous
Before we move on to another topic, there are three reasons why **recoding categorical data is one of the most dangerous tasks in data manipulation**.  

First, it is very easy here to mistakenly place the wrong labels on categories. If we make an error here, Python will not inform us of the error. It is important to read the codebook carefully and make certain that the categories are labeled correctly. For example, suppose I want to study the effect of attending a political protest on voters' preferences and beliefs. The ANES data contains a feature that reports whether or not each individual has attended a political protest in the last year. The value 1 means that the person attended a protest, and the value 2 means that the person did not attend a protest. If we mix up these labels, then any analysis that uses the protest feature will not only be wrong - it will generate results that are exactly the opposite of the truth. Suppose that people who attend a protest are 20% more likely to vote in the upcoming election than people who have not attended a protest. If we mislabel the protest categories, the result would suggest that people who attend a protest are 20% *less* likely to vote, and this result will have the same level of uncertainty as the truth. Imagine if these data came from a drug trial: if we mislabel the treatment and control groups, we would conclude that a drug that benefits patients harms those patients, or vice versa. 

Second, we have an opportunity to change the data in significant ways while recoding categories. These changes might be the correct ones to make, but it is important to be extremely transparent with all of the changes as they can impact the results of any subsequent analysis. For example, the party identification feature in the ANES data is coded as
* 1 - Strong Democrat
* 2 - Not very strong Democrat
* 3 - Independent, closer to Democrats
* 4 - Independent
* 5 - Independent, closer to Republicans
* 6 - Not very strong Republican
* 7 - Strong Republican

For my purposes I choose to recode this feature as having three categories, Democrat, Republican, and independent. But to do so, I have to choose how exactly to collapse categories. I choose to combine "Strong Democrat", "Not very strong Democrat", and "Independent, closer to Democrats" into a category named "Democrat", and "Strong Republican", "Not very strong Republican", and "Independent, closer to Republicans" into a category named "Republican". I leave "Independent" as it is. There is not an unambiguously correct way to perform this recoding: maybe the two leaning-independent categories should be grouped within "Independent" instead. I recode the feature in this way because I theorize that many people in the survey will declare themselves to be independent, but really behave more like Democrats or Republicans. Even though I have a reason for choosing my coding scheme, it is a decision that can change any analysis that uses party identification. Maybe the results would be very different if we expanded the independent category. 

Third, if the categories for a feature can be aligned in a meaningful order, we have a choice about whether to treat this feature as categorical or as numeric. If we treat the feature as categorical, then we label each number. That's useful especially for generating visualizations in which this feature comprises an axis. If we treat the feature as numeric, we leave the numbers as they are (while ensuring that the categories are in the right order). That's useful if we want to report statistics like the mean and variance, and we believe that these statistics have meaning for the ordered scale. For features like `universal_income` in the ANES data, the categories represent degrees of support for a policy. If we label the categories, we see clearly how many people adopt each nuanced position on the spectrum between support and non-support. If we leave the categories as numbers, we can report the average level of support on the 7-point scale. Whether to label the categories of an ordered categorical feature or leave them as numeric should therefore depend on the problems the feature will be applied to.

## Sorting Rows
Sorting is a largely cosmetic thing to do because most analyses don't use the ordering of the rows to generate results. That said, especially in the early data cleaning stages of an analysis, it can be very useful to sort the rows of a dataframe according to the values in one or more columns, especially if the data contain an ID with a clear alphabetic or numeric order, like country names or time.

To sort, use the `.sort_values()` method. Within the method, use the `by` argument to specify the column you want to sort by. By default, sorting is done in ascending order for numeric features and in alphabetical order for string features, To save the sorting, either write `inplace=True` or re-assign the sorted dataframe to the same variable. `.sort_values()` will always put rows that have missing values for the sorting column at the bottom of the dataframe. 

For example, to sort by lowest to highest ratings of Donald Trump, we can type:

In [39]:
anes_clean.sort_values(by = 'fttrump')

,caseid,liveurban,vote16,protest,vote,most_important_issue,confecon,ideology,partyID,universal_income,family_separation,free_college,forgive_loans,race,birthyr,sex,education,weight,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,ftimmig
3164,3165,Town,Hillary Clinton,False,Joe Biden,trump,Extremely worried,Moderate,Democrat,Oppose a great deal,Favor strongly,Oppose a great deal,Oppose a great deal,Other,1960,Male,High school graduate,.809576969671362,0.0,100.0,70.0,51.0,100.0,100.0,39.0,100.0,0.0,100.0,100.0,99.0,100.0,98,100.0,98.0,2.0,0.0,99,98.0,98.0,99.0,96.0,100.0,100,99,100,97.0,99.0,98,98.0,98.0,99.0,100.0
1761,1762,City,Hillary Clinton,False,Joe Biden,The current incumbent in the White House. He h...,Moderately worried,Liberal,Democrat,Neither favor nor oppose,Oppose strongly,Neither favor nor oppose,Oppose a little,White,1974,Male,4-year degree,1.03413132599488,0.0,74.0,67.0,63.0,80.0,41.0,52.0,90.0,90.0,90.0,90.0,90.0,80.0,55,60.0,50.0,1.0,0.0,5,0.0,84.0,0.0,62.0,22.0,70,70,65,30.0,58.0,99,38.0,6.0,56.0,89.0
419,420,City,Hillary Clinton,False,Joe Biden,Climate,Moderately worried,Moderate,Democrat,Favor a little,Oppose strongly,Favor a moderate amount,Favor a little,White,1960,Female,4-year degree,1.0029772835284,0.0,75.0,59.0,65.0,60.0,74.0,66.0,75.0,51.0,64.0,73.0,51.0,48.0,65,75.0,75.0,39.0,37.0,50,44.0,69.0,50.0,69.0,50.0,84,91,79,87.0,88.0,81,43.0,41.0,66.0,60.0
1763,1764,Suburb,Hillary Clinton,False,Joe Biden,Environment,Extremely worried,Liberal,Democrat,Favor a moderate amount,Oppose strongly,Favor a great deal,Favor a great deal,White,1980,Female,4-year degree,1.03404200778801,0.0,95.0,77.0,81.0,78.0,74.0,91.0,95.0,72.0,93.0,89.0,92.0,91.0,100,95.0,93.0,3.0,3.0,39,10.0,81.0,33.0,82.0,59.0,81,93,87,61.0,87.0,86,56.0,6.0,62.0,91.0
1767,1768,Town,Did not vote,True,Joe Biden,"Narcissism, sexism, homophobia, & global warming.",Moderately worried,NaN,NaN,Neither favor nor oppose,Oppose strongly,Neither favor nor oppose,Neither favor nor oppose,Hispanic,1991,Female,4-year degree,.818480694706408,0.0,10.0,71.0,22.0,72.0,54.0,67.0,26.0,97.0,59.0,99.0,57.0,72.0,82,52.0,52.0,1.0,48.0,52,50.0,4.0,49.0,50.0,3.0,7,51,49,48.0,53.0,98,51.0,49.0,50.0,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2764,2765,City,Did not vote,False,Someone else,make sure poeple dont do bad things,Moderately worried,NaN,Independent,Neither favor nor oppose,Neither favor nor disagree,Neither favor nor oppose,Neither favor nor oppose,White,1997,Female,High school graduate,1.08673596743961,NaN,100.0,NaN,NaN,NaN,NaN,NaN,12.0,100.0,6.0,16.0,39.0,19.0,31,28.0,36.0,72.0,53.0,22,45.0,21.0,29.0,49.0,44.0,10,100,41,39.0,47.0,31,10.0,46.0,26.0,27.0
2823,2824,Town,Did not vote,False,Donald Trump,Poverty,Very worried,NaN,Republican,Neither favor nor oppose,Neither favor nor disagree,Neither favor nor oppose,Neither favor nor oppose,White,1983,Female,2-year degree,.772007610334614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,9.0,6.0,7.0,4.0,6.0,6,NaN,NaN,NaN,NaN,6,5.0,4.0,5.0,6.0,6.0,6,8,7,6.0,3.0,9,6.0,3.0,6.0,2.0
2910,2911,City,Hillary Clinton,True,Joe Biden,,Very worried,Conservative,Democrat,Favor a moderate amount,Neither favor nor disagree,Neither favor nor oppose,Neither favor nor oppose,Black,1972,Female,High school graduate,.894064952261024,NaN,56.0,30.0,38.0,23.0,29.0,42.0,28.0,50.0,71.0,34.0,39.0,23.0,32,17.0,50.0,38.0,41.0,27,25.0,35.0,40.0,43.0,34.0,31,72,27,28.0,45.0,60,32.0,41.0,32.0,27.0
3019,3020,Town,Did not vote,False,Probably will not vote,Idk,Extremely worried,NaN,NaN,Neither favor nor oppose,Neither favor nor disagree,Neither favor nor oppose,Neither favor

To sort in descending order from the highest to lowest values of `fttrump`, we can use the `ascending=False` parameter:

In [40]:
anes_clean.sort_values(by = 'fttrump', ascending=False)

,caseid,liveurban,vote16,protest,vote,most_important_issue,confecon,ideology,partyID,universal_income,family_separation,free_college,forgive_loans,race,birthyr,sex,education,weight,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,ftimmig
2452,2453,Rural,Donald Trump,False,Donald Trump,None,A little worried,Conservative,Republican,Favor a little,Favor strongly,Neither favor nor oppose,Favor a little,White,1995,Female,High school graduate,.753518532246602,100.0,9.0,1.0,5.0,9.0,3.0,3.0,5.0,3.0,7.0,1.0,0.0,2.0,9,16.0,7.0,4.0,2.0,8,13.0,15.0,5.0,6.0,15.0,9,18,8,9.0,9.0,0,2.0,13.0,8.0,4.0
1735,1736,Suburb,Donald Trump,False,Donald Trump,The illegal immigrant entering the country ill...,A little worried,Conservative,Republican,Oppose a great deal,Favor somewhat,Oppose a great deal,Oppose a great deal,White,1981,Female,Some college,.819619380522467,100.0,3.0,0.0,0.0,1.0,0.0,4.0,54.0,53.0,50.0,48.0,3.0,0.0,2,1.0,1.0,99.0,97.0,0,1.0,0.0,0.0,0.0,3.0,53,9,25,88.0,52.0,2,52.0,2.0,2.0,100.0
3076,3077,Suburb,Donald Trump,False,Donald Trump,Fake News Media,A little worried,Conservative,Republican,Oppose a great deal,Neither favor nor disagree,Oppose a great deal,Oppose a great deal,White,1966,Male,Post-graduate,1.25802472009798,100.0,6.0,0.0,2.0,2.0,5.0,5.0,50.0,50.0,50.0,50.0,50.0,3.0,2,15.0,4.0,100.0,100.0,3,3.0,50.0,27.0,4.0,3.0,51,3,62,100.0,30.0,50,4.0,5.0,3.0,70.0
2174,2175,Rural,Donald Trump,False,Donald Trump,lifetime politicians,Not at all worried,Conservative,Republican,Oppose a great deal,Favor somewhat,Oppose a great deal,Oppose a great deal,White,1947,Male,High school graduate,,100.0,1.0,0.0,1.0,1.0,1.0,3.0,79.0,79.0,82.0,69.0,2.0,1.0,20,5.0,6.0,69.0,95.0,15,5.0,12.0,20.0,59.0,0.0,60,25,41,89.0,11.0,69,29.0,74.0,4.0,80.0
3077,3078,Suburb,Donald Trump,False,Donald Trump,Illegal immigration,Not at all worried,Conservative,Republican,Oppose a great deal,Neither favor nor disagree,Oppose a great deal,Oppose a great deal,White,1960,Male,Post-graduate,1.0152659648901,100.0,31.0,13.0,33.0,15.0,16.0,25.0,93.0,88.0,94.0,87.0,72.0,40.0,8,54.0,23.0,100.0,82.0,11,3.0,72.0,47.0,39.0,4.0,88,44,96,100.0,19.0,70,23.0,42.0,12.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2764,2765,City,Did not vote,False,Someone else,make sure poeple dont do bad things,Moderately worried,NaN,Independent,Neither favor nor oppose,Neither favor nor disagree,Neither favor nor oppose,Neither favor nor oppose,White,1997,Female,High school graduate,1.08673596743961,NaN,100.0,NaN,NaN,NaN,NaN,NaN,12.0,100.0,6.0,16.0,39.0,19.0,31,28.0,36.0,72.0,53.0,22,45.0,21.0,29.0,49.0,44.0,10,100,41,39.0,47.0,31,10.0,46.0,26.0,27.0
2823,2824,Town,Did not vote,False,Donald Trump,Poverty,Very worried,NaN,Republican,Neither favor nor oppose,Neither favor nor disagree,Neither favor nor oppose,Neither favor nor oppose,White,1983,Female,2-year degree,.772007610334614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,9.0,6.0,7.0,4.0,6.0,6,NaN,NaN,NaN,NaN,6,5.0,4.0,5.0,6.0,6.0,6,8,7,6.0,3.0,9,6.0,3.0,6.0,2.0
2910,2911,City,Hillary Clinton,True,Joe Biden,,Very worried,Conservative,Democrat,Favor a moderate amount,Neither favor nor disagree,Neither favor nor oppose,Neither favor nor oppose,Black,1972,Female,High school graduate,.894064952261024,NaN,56.0,30.0,38.0,23.0,29.0,42.0,28.0,50.0,71.0,34.0,39.0,23.0,32,17.0,50.0,38.0,41.0,27,25.0,35.0,40.0,43.0,34.0,31,72,27,28.0,45.0,60,32.0,41.0,32.0,27.0
3019,3020,Town,Did not vote,False,Probably will not vote,Idk,Extremely worried,NaN,NaN,Neither favor nor oppose,Neither favor nor disagree,Neither favor nor oppose,Neither favor nor oppose,Black,1994,Female,2-year degree,1

We can also sort the rows by the values of more than one column. When we do that, the second (and third, and so on) column is only used to break the ties of the first (and second, etc.) column. All we have to do is write more than one column name in a list, and pass that list to the `by` argument of `.sort_values()`. The `ascending` argument also takes a list of `True` or `False` values to denote whether each of the columns should be sorted in ascending or descending order. To sort the rows in ascending order by values of the Donald Trump thermometer, while breaking ties by descending order of the Joe Biden thermometer, we can type:

In [41]:
anes_clean.sort_values(by=['fttrump', 'ftbiden'], ascending = [True, False])

,caseid,liveurban,vote16,protest,vote,most_important_issue,confecon,ideology,partyID,universal_income,family_separation,free_college,forgive_loans,race,birthyr,sex,education,weight,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,ftimmig
94,95,Suburb,Hillary Clinton,False,Joe Biden,The president.,Not at all worried,Liberal,Democrat,Neither favor nor oppose,Oppose strongly,Favor a moderate amount,Favor a great deal,White,1947,Female,Some college,1.02094882538262,0.0,100.0,100.0,71.0,70.0,99.0,60.0,50.0,50.0,50.0,51.0,50.0,50.0,85,99.0,94.0,25.0,0.0,0,0.0,96.0,56.0,95.0,0.0,100,94,95,41.0,95.0,100,30.0,0.0,69.0,50.0
262,263,City,Hillary Clinton,False,Joe Biden,Trumps presidency,Moderately worried,Liberal,Democrat,Favor a little,Oppose strongly,Favor a moderate amount,Oppose a little,White,1959,Male,High school graduate,.878855634357939,0.0,99.0,100.0,91.0,100.0,91.0,41.0,50.0,50.0,91.0,50.0,40.0,50.0,90,99.0,87.0,2.0,1.0,4,2.0,80.0,0.0,51.0,1.0,96,91,91,40.0,99.0,99,1.0,1.0,91.0,81.0
355,356,Suburb,Hillary Clinton,False,Joe Biden,That Donald Trump is an immoral racist man who...,A little worried,Moderate,Democrat,Oppose a great deal,Oppose strongly,Favor a great deal,Favor a great deal,White,1957,Female,2-year degree,1.03170391458116,0.0,80.0,100.0,27.0,28.0,98.0,31.0,50.0,52.0,51.0,50.0,51.0,71.0,74,66.0,57.0,6.0,1.0,5,1.0,70.0,2.0,51.0,2.0,99,100,86,38.0,78.0,99,3.0,2.0,NaN,74.0
461,462,Town,Hillary Clinton,False,Joe Biden,The division made by Donald Trump as it concer...,Moderately worried,Liberal,Democrat,Oppose a moderate amount,Oppose strongly,Favor a little,Favor a moderate amount,White,1955,Female,Some college,1.03170391458116,0.0,91.0,100.0,70.0,80.0,91.0,51.0,89.0,80.0,91.0,90.0,88.0,71.0,96,100.0,90.0,4.0,5.0,37,4.0,79.0,28.0,84.0,33.0,100,85,90,80.0,85.0,98,40.0,5.0,79.0,90.0
499,500,City,Hillary Clinton,False,Joe Biden,The economy,Extremely worried,NaN,Democrat,Favor a great deal,Oppose strongly,Favor a moderate amount,Favor a great deal,Hispanic,1967,Male,No HS diploma,2.26361888055549,0.0,98.0,100.0,98.0,97.0,NaN,NaN,66.0,99.0,100.0,100.0,97.0,99.0,68,NaN,99.0,2.0,39.0,100,2.0,55.0,16.0,99.0,3.0,99,100,58,97.0,100.0,99,63.0,13.0,99.0,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,2500,City,Did not vote,False,Probably will not vote,don’t know,A little worried,NaN,NaN,Neither favor nor oppose,Neither favor nor disagree,Neither favor nor oppose,Neither favor nor oppose,Hispanic,1998,Female,Some college,.893556010098802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,50.0,50.0,51.0,52.0,49.0,51,NaN,NaN,NaN,NaN,50,50.0,51.0,51.0,47.0,49.0,51,48,50,50.0,50.0,50,53.0,52.0,48.0,48.0
2533,2534,Suburb,Did not vote,False,Probably will not vote,Everything,A little worried,NaN,NaN,Neither favor nor oppose,Neither favor nor disagree,Neither favor nor oppose,Neither favor nor oppose,Black,1987,Female,High school graduate,.738770525410191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0,75.0,53.0,13.0,88.0,47.0,88,NaN,13.0,NaN,NaN,6,7.0,4.0,2.0,11.0,6.0,7,3,4,1.0,5.0,3,9.0,6.0,6.0,48.0
2764,2765,City,Did not vote,False,Someone else,make sure poeple dont do bad things,Moderately worried,NaN,Independent,Neither favor nor oppose,Neither favor nor disagree,Neither favor nor oppose,Neither favor nor oppose,White,1997,Female,High school graduate,1.08673596743961,NaN,100.0,NaN,NaN,NaN,NaN,NaN,12.0,100.0,6.0,16.0,39.0,19.0,31,28.0,36.0,72.0,53.0,22,45.0,21.0,29.0,49.0,44.0,10,100,41,39.0,47.0,31,10.0,46.0,26.0,27.0
2823,2824,Town,Did not vote,False,Donald Trump,Poverty,Very worried,NaN,Republican,Neither favor nor oppose,Neither favor nor disagree,Neither favor nor oppo

## Filtering Rows
Filtering is the task of keeping or deleting rows based on a logical condition. Prior to discussing the code for filtering, let's review logical operators. Every logical statement returns a value of `True` or `False` in Python, and these statements are useful for keeping rows for which a condition is `True` and deleting rows for which a condition is `False`. 

Python has the following logical operators:

* `==` "is equal to"

* `!=` "is not equal to"

* `>` "is greater than"

* `>=` "is greater than or equal to"

* `<` "is less than"

* `<=` "is less than or equal to"

* `in` "is in the list"

* `not in` "is not in the list"

It also has the following symbols to *connect* logical statements to build more complicated statements:

* `&` "and" (both conditions must be true for the whole statement to be true)

* `|` "or" (at least one of the conditiona must be true for the whole statement to be true)

* `not` turns `True` results to `False`, and vice versa

* `(` and `)` (parentheses work with logic the way they do with algebra -- consider this part of the statement first)

There are many ways to filter the rows in a dataframe based on a logical condition, but the best method is `.query()` because of its notational ease and because it allows **chaining**: that is, specifying many logical conditions with repeated calls to `.query()`. Chaining is another way to express "and" in a combined logical statement. To use `.query()`, type it as a method applied to the dataframe in question, and put the logical condition in double quotes - that allows us to use single quotes within the query. For example, to see just the Joe Biden voters, type:

In [42]:
anes_clean.query("vote=='Joe Biden'")

,caseid,liveurban,vote16,protest,vote,most_important_issue,confecon,ideology,partyID,universal_income,family_separation,free_college,forgive_loans,race,birthyr,sex,education,weight,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,ftimmig
0,1,Suburb,Someone else,False,Joe Biden,Health Care,A little worried,Conservative,Democrat,Favor a moderate amount,Oppose strongly,Favor a moderate amount,Favor a little,Hispanic,1969,Male,High school graduate,1.34719693063187,47.0,90.0,52.0,52.0,49.0,NaN,50.0,99.0,99.0,99.0,100.0,88.0,79.0,99,82.0,71.0,86.0,88.0,90,66.0,89.0,88.0,81.0,77.0,98,94,89,88.0,99.0,99,92.0,89.0,86.0,97.0
2,3,Rural,Hillary Clinton,False,Joe Biden,health care,Extremely worried,Moderate,Democrat,Neither favor nor oppose,Oppose strongly,Oppose a moderate amount,Oppose a great deal,White,1954,Female,High school graduate,.966366930694957,0.0,91.0,88.0,15.0,60.0,70.0,68.0,48.0,49.0,49.0,49.0,50.0,39.0,63,66.0,51.0,40.0,2.0,2,2.0,1.0,3.0,59.0,1.0,50,1,1,1.0,51.0,87,50.0,1.0,3.0,69.0
5,6,Suburb,Hillary Clinton,False,Joe Biden,The influence of big money on our political sy...,Not at all worried,Moderate,Democrat,Favor a moderate amount,Oppose strongly,Favor a moderate amount,Favor a great deal,Black,1942,Male,4-year degree,1.02140871415171,46.0,83.0,80.0,69.0,80.0,59.0,64.0,87.0,69.0,80.0,61.0,70.0,63.0,56,68.0,70.0,58.0,43.0,61,47.0,76.0,51.0,81.0,49.0,81,65,74,95.0,72.0,71,51.0,44.0,81.0,81.0
6,7,City,Did not vote,False,Joe Biden,,Moderately worried,Liberal,Democrat,Neither favor nor oppose,Oppose strongly,Neither favor nor oppose,Neither favor nor oppose,White,2000,Female,Some college,.964514474045239,0.0,100.0,75.0,91.0,52.0,71.0,81.0,98.0,85.0,97.0,98.0,98.0,73.0,70,97.0,97.0,47.0,3.0,53,51.0,75.0,50.0,70.0,50.0,59,74,73,50.0,90.0,95,63.0,18.0,70.0,99.0
7,8,City,Hillary Clinton,True,Joe Biden,climate change,Not at all worried,Liberal,Democrat,Favor a little,Oppose strongly,Favor a little,Oppose a moderate amount,White,1994,Female,Post-graduate,.83469258858232,1.0,83.0,45.0,86.0,71.0,71.0,79.0,73.0,75.0,76.0,76.0,79.0,75.0,75,50.0,86.0,40.0,1.0,23,0.0,70.0,38.0,50.0,35.0,60,66,78,56.0,58.0,82,50.0,33.0,46.0,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3153,3154,City,Donald Trump,False,Joe Biden,The manner in which Donald Trump conducts hims...,Moderately worried,Moderate,Democrat,Favor a great deal,Oppose strongly,Oppose a great deal,Oppose a great deal,White,1939,Male,Some college,.83972997938151,6.0,69.0,12.0,20.0,50.0,99.0,13.0,69.0,69.0,69.0,69.0,69.0,31.0,59,51.0,51.0,79.0,1.0,15,2.0,48.0,3.0,54.0,4.0,83,72,70,50.0,75.0,84,13.0,9.0,50.0,69.0
3154,3155,Town,Did not vote,False,Joe Biden,trump,Moderately worried,Liberal,Democrat,Favor a great deal,Oppose strongly,Favor a great deal,Favor a great deal,White,1978,Male,4-year degree,.919903784560563,1.0,81.0,61.0,65.0,81.0,70.0,65.0,50.0,60.0,46.0,56.0,45.0,50.0,60,70.0,60.0,30.0,6.0,35,21.0,41.0,11.0,50.0,11.0,69,45,75,46.0,56.0,85,27.0,15.0,36.0,55.0
3156,3157,Suburb,Hillary Clinton,False,Joe Biden,health care,A little worried,Moderate,Democrat,Favor a moderate amount,Oppose strongly,Oppose a moderate amount,Oppose a great deal,White,1980,Male,Post-graduate,.783602487218187,1.0,95.0,86.0,41.0,41.0,81.0,70.0,99.0,99.0,100.0,100.0,45.0,60.0,100,94.0,95.0,46.0,16.0,79,30.0,99.0,48.0,65.0,28.0,85,99,100,66.0,99.0,99,40.0,70.0,56.0,100.0
3163,3164,Suburb,Did not vote,False,Joe Biden,Donald Trump,Moderately worried,Liberal,Democrat,Favor a moderate amount,Oppose somewhat,Neither favor nor oppose,Oppose a little,Black,1960,Male,No HS diploma,1.58161278448241,1.0,100.0,95.0,62.0,79.0

To keep the rows for people that will vote for Joe Biden, but rate him less than 40:

In [43]:
anes_clean.query("vote=='Joe Biden' & ftbiden < 40")

,caseid,liveurban,vote16,protest,vote,most_important_issue,confecon,ideology,partyID,universal_income,family_separation,free_college,forgive_loans,race,birthyr,sex,education,weight,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,ftimmig
12,13,City,Donald Trump,False,Joe Biden,Assumenda mollit bla,Extremely worried,Moderate,Independent,Favor a moderate amount,Favor somewhat,Oppose a little,Neither favor nor oppose,White,1994,Female,2-year degree,1.08428147456979,25.0,66.0,31.0,86.0,NaN,53.0,78.0,69.0,21.0,51.0,47.0,70.0,54.0,62,49.0,NaN,NaN,NaN,65,42.0,26.0,33.0,51.0,53.0,78,61,46,76.0,38.0,36,10.0,67.0,50.0,34.0
63,64,Suburb,Hillary Clinton,False,Joe Biden,Climate change and partly because we refuse to...,A little worried,Liberal,Democrat,Oppose a little,Neither favor nor disagree,Oppose a little,Favor a little,White,1930,Female,Post-graduate,1.02996454702489,5.0,76.0,37.0,73.0,64.0,55.0,39.0,67.0,64.0,71.0,67.0,59.0,64.0,66,63.0,50.0,22.0,6.0,21,5.0,50.0,50.0,22.0,26.0,68,59,60,31.0,50.0,87,44.0,22.0,71.0,74.0
65,66,Rural,Hillary Clinton,False,Joe Biden,Trump,Moderately worried,Liberal,Democrat,Neither favor nor oppose,Oppose strongly,Oppose a little,Oppose a moderate amount,White,1952,Female,High school graduate,1.02515128827396,0.0,100.0,33.0,34.0,96.0,96.0,42.0,90.0,49.0,21.0,50.0,50.0,0.0,71,50.0,51.0,3.0,0.0,53,0.0,22.0,0.0,61.0,1.0,80,50,30,1.0,53.0,94,12.0,0.0,10.0,76.0
105,106,Town,Did not vote,False,Joe Biden,guns,Moderately worried,NaN,NaN,Neither favor nor oppose,Oppose strongly,Favor a little,Favor a great deal,Other,1996,Female,4-year degree,.77435239235677,0.0,51.0,8.0,59.0,73.0,76.0,2.0,53.0,0.0,44.0,87.0,3.0,50.0,52,8.0,33.0,5.0,0.0,4,50.0,2.0,1.0,38.0,37.0,5,51,66,1.0,6.0,30,100.0,2.0,47.0,50.0
129,130,Town,Hillary Clinton,False,Joe Biden,"Lack of access to healthcare, racism, white na...",Very worried,Liberal,Democrat,Favor a great deal,Oppose strongly,Favor a great deal,Favor a great deal,White,1977,Female,Post-graduate,1.15605850208199,0.0,61.0,9.0,90.0,100.0,15.0,42.0,100.0,64.0,95.0,100.0,100.0,79.0,89,71.0,90.0,0.0,0.0,59,0.0,69.0,6.0,50.0,59.0,88,100,86,27.0,100.0,91,15.0,22.0,83.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3063,3064,Rural,Hillary Clinton,False,Joe Biden,Gun violence,A little worried,Liberal,Democrat,Oppose a moderate amount,Oppose strongly,Favor a great deal,Favor a moderate amount,White,1957,Male,High school graduate,.897550307642941,5.0,65.0,21.0,71.0,84.0,34.0,59.0,69.0,59.0,50.0,46.0,39.0,36.0,59,50.0,50.0,50.0,5.0,53,0.0,61.0,29.0,18.0,18.0,97,69,67,59.0,52.0,89,39.0,6.0,17.0,50.0
3069,3070,Suburb,Hillary Clinton,False,Joe Biden,ECONOMY,Very worried,Liberal,Democrat,Favor a moderate amount,Oppose somewhat,Favor a great deal,Favor a moderate amount,White,1983,Male,2-year degree,.938881851047225,11.0,71.0,31.0,39.0,99.0,30.0,19.0,30.0,51.0,71.0,94.0,47.0,28.0,51,50.0,51.0,59.0,49.0,61,59.0,69.0,50.0,50.0,50.0,50,69,70,50.0,59.0,60,49.0,51.0,51.0,69.0
3136,3137,Rural,Hillary Clinton,False,Joe Biden,Donald J. Trump,Moderately worried,Liberal,Democrat,Favor a little,Oppose strongly,Favor a little,Favor a little,White,1950,Male,Some college,.944411233915414,3.0,86.0,22.0,94.0,9.0,94.0,25.0,53.0,58.0,59.0,57.0,59.0,60.0,94,84.0,85.0,11.0,3.0,5,6.0,94.0,5.0,88.0,6.0,92,84,84,76.0,88.0,95,5.0,4.0,53.0,67.0
3146,3147,Suburb,Hillary Clinton,False,Joe Biden,The ever-growing populism from both sides of t...,Extremely worried,Liberal,Democrat,Favor a great deal,Oppose strongly,Favor a moderate amount,Favor a great deal,White,1995,Male,4-year degree,1.08737874806145,0.0,76.0,34.0,87.0,22.0,66.0,

Or alternatively,

In [44]:
anes_clean.query("vote=='Joe Biden'").query('ftbiden < 40')

,caseid,liveurban,vote16,protest,vote,most_important_issue,confecon,ideology,partyID,universal_income,family_separation,free_college,forgive_loans,race,birthyr,sex,education,weight,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,ftimmig
12,13,City,Donald Trump,False,Joe Biden,Assumenda mollit bla,Extremely worried,Moderate,Independent,Favor a moderate amount,Favor somewhat,Oppose a little,Neither favor nor oppose,White,1994,Female,2-year degree,1.08428147456979,25.0,66.0,31.0,86.0,NaN,53.0,78.0,69.0,21.0,51.0,47.0,70.0,54.0,62,49.0,NaN,NaN,NaN,65,42.0,26.0,33.0,51.0,53.0,78,61,46,76.0,38.0,36,10.0,67.0,50.0,34.0
63,64,Suburb,Hillary Clinton,False,Joe Biden,Climate change and partly because we refuse to...,A little worried,Liberal,Democrat,Oppose a little,Neither favor nor disagree,Oppose a little,Favor a little,White,1930,Female,Post-graduate,1.02996454702489,5.0,76.0,37.0,73.0,64.0,55.0,39.0,67.0,64.0,71.0,67.0,59.0,64.0,66,63.0,50.0,22.0,6.0,21,5.0,50.0,50.0,22.0,26.0,68,59,60,31.0,50.0,87,44.0,22.0,71.0,74.0
65,66,Rural,Hillary Clinton,False,Joe Biden,Trump,Moderately worried,Liberal,Democrat,Neither favor nor oppose,Oppose strongly,Oppose a little,Oppose a moderate amount,White,1952,Female,High school graduate,1.02515128827396,0.0,100.0,33.0,34.0,96.0,96.0,42.0,90.0,49.0,21.0,50.0,50.0,0.0,71,50.0,51.0,3.0,0.0,53,0.0,22.0,0.0,61.0,1.0,80,50,30,1.0,53.0,94,12.0,0.0,10.0,76.0
105,106,Town,Did not vote,False,Joe Biden,guns,Moderately worried,NaN,NaN,Neither favor nor oppose,Oppose strongly,Favor a little,Favor a great deal,Other,1996,Female,4-year degree,.77435239235677,0.0,51.0,8.0,59.0,73.0,76.0,2.0,53.0,0.0,44.0,87.0,3.0,50.0,52,8.0,33.0,5.0,0.0,4,50.0,2.0,1.0,38.0,37.0,5,51,66,1.0,6.0,30,100.0,2.0,47.0,50.0
129,130,Town,Hillary Clinton,False,Joe Biden,"Lack of access to healthcare, racism, white na...",Very worried,Liberal,Democrat,Favor a great deal,Oppose strongly,Favor a great deal,Favor a great deal,White,1977,Female,Post-graduate,1.15605850208199,0.0,61.0,9.0,90.0,100.0,15.0,42.0,100.0,64.0,95.0,100.0,100.0,79.0,89,71.0,90.0,0.0,0.0,59,0.0,69.0,6.0,50.0,59.0,88,100,86,27.0,100.0,91,15.0,22.0,83.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3063,3064,Rural,Hillary Clinton,False,Joe Biden,Gun violence,A little worried,Liberal,Democrat,Oppose a moderate amount,Oppose strongly,Favor a great deal,Favor a moderate amount,White,1957,Male,High school graduate,.897550307642941,5.0,65.0,21.0,71.0,84.0,34.0,59.0,69.0,59.0,50.0,46.0,39.0,36.0,59,50.0,50.0,50.0,5.0,53,0.0,61.0,29.0,18.0,18.0,97,69,67,59.0,52.0,89,39.0,6.0,17.0,50.0
3069,3070,Suburb,Hillary Clinton,False,Joe Biden,ECONOMY,Very worried,Liberal,Democrat,Favor a moderate amount,Oppose somewhat,Favor a great deal,Favor a moderate amount,White,1983,Male,2-year degree,.938881851047225,11.0,71.0,31.0,39.0,99.0,30.0,19.0,30.0,51.0,71.0,94.0,47.0,28.0,51,50.0,51.0,59.0,49.0,61,59.0,69.0,50.0,50.0,50.0,50,69,70,50.0,59.0,60,49.0,51.0,51.0,69.0
3136,3137,Rural,Hillary Clinton,False,Joe Biden,Donald J. Trump,Moderately worried,Liberal,Democrat,Favor a little,Oppose strongly,Favor a little,Favor a little,White,1950,Male,Some college,.944411233915414,3.0,86.0,22.0,94.0,9.0,94.0,25.0,53.0,58.0,59.0,57.0,59.0,60.0,94,84.0,85.0,11.0,3.0,5,6.0,94.0,5.0,88.0,6.0,92,84,84,76.0,88.0,95,5.0,4.0,53.0,67.0
3146,3147,Suburb,Hillary Clinton,False,Joe Biden,The ever-growing populism from both sides of t...,Extremely worried,Liberal,Democrat,Favor a great deal,Oppose strongly,Favor a moderate amount,Favor a great deal,White,1995,Male,4-year degree,1.08737874806145,0.0,76.0,34.0,87.0,22.0,66.0,

Who in their right mind would rate Joe Biden 100 and Trump 100? These 6 people!

In [45]:
anes_clean.query("ftbiden == 100 & fttrump == 100")

,caseid,liveurban,vote16,protest,vote,most_important_issue,confecon,ideology,partyID,universal_income,family_separation,free_college,forgive_loans,race,birthyr,sex,education,weight,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,ftimmig
267,268,City,Donald Trump,False,Donald Trump,Education.,A little worried,Conservative,Republican,Favor a little,Oppose somewhat,Favor a little,Favor a great deal,White,1959,Female,Post-graduate,.746904935636401,100.0,0.0,100.0,1.0,4.0,NaN,NaN,70.0,90.0,61.0,63.0,15.0,6.0,21,48.0,36.0,49.0,43.0,50,50.0,43.0,39.0,60.0,43.0,58,40,63,90.0,69.0,53,53.0,56.0,54.0,16.0
1526,1527,City,Donald Trump,False,Donald Trump,EVERYTHING,Not at all worried,Conservative,Republican,Favor a great deal,Neither favor nor disagree,Neither favor nor oppose,Neither favor nor oppose,White,1956,Male,High school graduate,.847649185791497,100.0,99.0,100.0,71.0,81.0,80.0,90.0,99.0,100.0,95.0,91.0,82.0,50.0,98,94.0,89.0,96.0,99.0,71,70.0,78.0,68.0,69.0,47.0,85,77,83,57.0,100.0,100,91.0,88.0,76.0,99.0
1971,1972,Rural,Did not vote,False,Joe Biden,Proverty,Moderately worried,NaN,Republican,Neither favor nor oppose,Oppose strongly,Favor a great deal,Favor a great deal,White,1993,Female,2-year degree,.872100489544827,100.0,99.0,100.0,66.0,100.0,60.0,57.0,100.0,100.0,100.0,100.0,100.0,100.0,53,100.0,67.0,100.0,100.0,100,100.0,100.0,99.0,100.0,100.0,100,100,98,100.0,100.0,100,100.0,98.0,100.0,100.0
2271,2272,Town,Hillary Clinton,False,Someone else,jobs and homeless and Police just shoting norm...,Extremely worried,Conservative,Democrat,Favor a great deal,Favor strongly,Favor a great deal,Favor a great deal,White,1980,Male,Some college,1.05546596271457,100.0,78.0,100.0,58.0,29.0,5.0,100.0,90.0,94.0,89.0,9.0,88.0,89.0,86,86.0,79.0,79.0,69.0,68,27.0,57.0,7.0,9.0,8.0,68,10,77,5.0,8.0,62,95.0,9.0,78.0,88.0
2408,2409,City,Did not vote,False,Joe Biden,Not being the best number one country.,Extremely worried,Moderate,Independent,Favor a great deal,Favor strongly,Oppose a great deal,Favor a great deal,Black,1995,Male,Some college,3.47027368867645,100.0,100.0,100.0,99.0,100.0,99.0,100.0,100.0,100.0,100.0,99.0,75.0,2.0,100,100.0,100.0,99.0,99.0,100,100.0,100.0,100.0,100.0,100.0,100,100,100,100.0,100.0,100,100.0,100.0,100.0,100.0
2721,2722,Rural,Hillary Clinton,True,Joe Biden,Racism,Extremely worried,Liberal,Democrat,Favor a great deal,Favor strongly,Favor a great deal,Favor a great deal,Black,1990,Male,High school graduate,2.13773400773957,100.0,98.0,100.0,98.0,99.0,79.0,99.0,100.0,98.0,85.0,82.0,84.0,94.0,90,91.0,97.0,97.0,92.0,88,92.0,97.0,94.0,94.0,88.0,92,93,94,86.0,96.0,88,93.0,94.0,89.0,93.0


I want to see all of the voters that are extremely worried about the economy and either live in rural areas or were born prior to 1950:

In [46]:
anes_clean.query("confecon == 'Extremely worried' & (liveurban == 'Rural' | birthyr < 1950)")

,caseid,liveurban,vote16,protest,vote,most_important_issue,confecon,ideology,partyID,universal_income,family_separation,free_college,forgive_loans,race,birthyr,sex,education,weight,fttrump,ftobama,ftbiden,ftwarren,ftsanders,ftbuttigieg,ftharris,ftblack,ftwhite,fthisp,ftasian,ftmuslim,ftillegal,ftjournal,ftnato,ftun,ftice,ftnra,ftchina,ftnkorea,ftmexico,ftsaudi,ftukraine,ftiran,ftbritain,ftgermany,ftjapan,ftisrael,ftfrance,ftcanada,ftturkey,ftrussia,ftpales,ftimmig
2,3,Rural,Hillary Clinton,False,Joe Biden,health care,Extremely worried,Moderate,Democrat,Neither favor nor oppose,Oppose strongly,Oppose a moderate amount,Oppose a great deal,White,1954,Female,High school graduate,.966366930694957,0.0,91.0,88.0,15.0,60.0,70.0,68.0,48.0,49.0,49.0,49.0,50.0,39.0,63,66.0,51.0,40.0,2.0,2,2.0,1.0,3.0,59.0,1.0,50,1,1,1.0,51.0,87,50.0,1.0,3.0,69.0
27,28,Town,Hillary Clinton,False,Joe Biden,Climate change,Extremely worried,Liberal,Democrat,Favor a great deal,Oppose strongly,Favor a moderate amount,Favor a great deal,Other,1944,Female,4-year degree,1.01184109114119,2.0,91.0,49.0,96.0,100.0,87.0,76.0,51.0,52.0,50.0,46.0,45.0,62.0,99,99.0,49.0,0.0,0.0,31,6.0,73.0,7.0,64.0,56.0,92,85,78,25.0,87.0,99,38.0,3.0,52.0,96.0
72,73,Rural,Hillary Clinton,False,Joe Biden,Climate change,Extremely worried,Liberal,Democrat,Favor a little,Oppose strongly,Favor a great deal,Favor a great deal,White,1950,Female,Post-graduate,1.12291843396482,0.0,99.0,97.0,91.0,81.0,100.0,99.0,89.0,62.0,94.0,83.0,89.0,99.0,99,98.0,100.0,11.0,6.0,37,3.0,100.0,2.0,67.0,6.0,100,96,100,98.0,100.0,99,2.0,3.0,91.0,97.0
117,118,Rural,Did not vote,False,Joe Biden,Holding on to our Democracy is our most pressi...,Extremely worried,Moderate,Democrat,Favor a great deal,Oppose strongly,Favor a moderate amount,Favor a great deal,Black,1949,Male,2-year degree,1.04281941711299,50.0,98.0,96.0,91.0,90.0,88.0,86.0,91.0,90.0,90.0,90.0,87.0,89.0,95,100.0,100.0,60.0,50.0,77,50.0,91.0,50.0,91.0,72.0,93,85,83,81.0,94.0,91,71.0,40.0,83.0,91.0
149,150,Suburb,Hillary Clinton,False,Joe Biden,Infrastructure,Extremely worried,Moderate,Democrat,Oppose a great deal,Oppose strongly,Oppose a great deal,Oppose a great deal,White,1947,Male,Post-graduate,.826789085004023,0.0,98.0,91.0,1.0,3.0,95.0,1.0,60.0,72.0,68.0,81.0,41.0,0.0,90,90.0,69.0,6.0,0.0,0,0.0,59.0,0.0,50.0,0.0,100,99,50,31.0,60.0,100,0.0,0.0,10.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3023,3024,Rural,Hillary Clinton,False,Joe Biden,Imagation,Extremely worried,Conservative,Democrat,Neither favor nor oppose,Favor strongly,Oppose a great deal,Oppose a great deal,White,1962,Male,High school graduate,.878855634357939,49.0,85.0,68.0,66.0,97.0,49.0,54.0,3.0,100.0,5.0,2.0,2.0,2.0,1,99.0,100.0,99.0,99.0,98,0.0,95.0,6.0,9.0,98.0,70,94,96,3.0,6.0,97,1.0,97.0,0.0,1.0
3025,3026,Rural,Hillary Clinton,False,Joe Biden,The presidency of Donald J. Trump.,Extremely worried,Moderate,Democrat,Favor a little,Oppose strongly,Oppose a great deal,Oppose a great deal,Black,1960,Male,High school graduate,.989348149771041,0.0,100.0,90.0,75.0,70.0,55.0,75.0,60.0,49.0,50.0,50.0,60.0,49.0,50,70.0,75.0,50.0,0.0,50,0.0,49.0,20.0,54.0,50.0,49,50,50,40.0,65.0,70,40.0,0.0,85.0,50.0
3029,3030,Rural,Hillary Clinton,False,Joe Biden,racism hate crimes,Extremely worried,Liberal,Democrat,Neither favor nor oppose,Oppose strongly,Favor a great deal,Favor a moderate amount,White,1951,Male,Some college,.98338714821771,4.0,99.0,81.0,99.0,92.0,80.0,86.0,83.0,89.0,75.0,71.0,75.0,64.0,72,85.0,76.0,6.0,4.0,56,26.0,59.0,31.0,9.0,16.0,85,94,72,69.0,83.0,94,26.0,5.0,31.0,74.0
3103,3104,Town,Donald Trump,False,Joe Biden,climate change,Extremely worried,Liberal,Republican,Favor a little,Oppose strongly,Favor a little,Favor a little,White,1944,Male,4-year degree,,3.0,30.0,96.0,97.0,100.0,96.0,51.0,97.0,97.0,97.0,95.0,96.0,99.0,91

## Creating New Columns and Replacing Existing Columns
### Using Arithmetic Operations
There are many situations in which it makes sense to create new columns based on calculations with existing columns. There are two methods for  creating a new column or replace an existing one. The first approach is to define a new column name in a dataframe's index, and assign it to new values constructed from existing values in the dataframe.

For example, we can create a column that represents the difference between the Biden thermometer and the Trump thermometer, so that negative values (up to -100) represent partisanship for Trump, and positive values (up to 100) represent partisanship for Biden. We can either create a new column that contains these differences, or we can replace an existing column with this new column. To create a new column called `partisanship`, we can type:

In [47]:
anes_clean['partisanship'] = anes_clean.ftbiden - anes_clean.fttrump
anes_clean[['ftbiden', 'fttrump', 'partisanship']]

,ftbiden,fttrump,partisanship
0,52.0,47.0,5.0
1,41.0,41.0,0.0
2,88.0,0.0,88.0
3,0.0,100.0,-100.0
4,25.0,94.0,-69.0
...,...,...,...
3160,7.0,81.0,-74.0
3161,25.0,35.0,-10.0
3162,50.0,6.0,44.0
3163,95.0,1.0,94.0


The second approach is to use the dataframe's `.assign()` method. Within `assign()`, we write the name of the column we are creating, and we set it equal to an arithmetic or logical expression involving existing columns. The equivalent code that creates a `partisanship` column using the `.assign()` method is:

In [48]:
anes_clean = anes_clean.assign(partisanship = anes_clean.ftbiden - anes_clean.fttrump)
anes_clean[['ftbiden', 'fttrump', 'partisanship']]

,ftbiden,fttrump,partisanship
0,52.0,47.0,5.0
1,41.0,41.0,0.0
2,88.0,0.0,88.0
3,0.0,100.0,-100.0
4,25.0,94.0,-69.0
...,...,...,...
3160,7.0,81.0,-74.0
3161,25.0,35.0,-10.0
3162,50.0,6.0,44.0
3163,95.0,1.0,94.0


To replace an existing column with a new version of that column, we use the same syntax, but we write an existing column name to set equal to an expression. For example, to replace the Barack Obama feeling thermometer with a Z-score standardized version of this column (subtracting the mean and dividing by the standard deviation) so that the values of `ftobama` change to represent the number of standard deviations away from the mean of `ftobama`. We can type

In [49]:
obama_mean = anes_clean.ftobama.mean()
obama_sd = anes_clean.ftobama.std()
anes_clean['ftobama'] = (anes_clean.ftobama - obama_mean)/obama_sd
anes_clean[['ftbiden', 'fttrump', 'ftobama']]

,ftbiden,fttrump,ftobama
0,52.0,47.0,0.971498
1,41.0,41.0,-0.626842
2,88.0,0.0,0.998137
3,0.0,100.0,-0.094062
4,25.0,94.0,-0.946510
...,...,...,...
3160,7.0,81.0,-0.893232
3161,25.0,35.0,-0.333813
3162,50.0,6.0,-0.600203
3163,95.0,1.0,1.237888


Then we can reconstruct the original `ftobama` feature:

In [50]:
anes_clean['ftobama'] = anes_clean.ftobama*obama_sd + obama_mean
anes_clean[['ftbiden', 'fttrump', 'ftobama']]

,ftbiden,fttrump,ftobama
0,52.0,47.0,90.0
1,41.0,41.0,30.0
2,88.0,0.0,91.0
3,0.0,100.0,50.0
4,25.0,94.0,18.0
...,...,...,...
3160,7.0,81.0,20.0
3161,25.0,35.0,41.0
3162,50.0,6.0,31.0
3163,95.0,1.0,100.0


We can also perform both tasks using `.assign()`. We can replace `ftobama` with a standardized version of the feature:

In [51]:
anes_clean = anes_clean.assign(ftobama = (anes_clean.ftobama - obama_mean)/obama_sd)
anes_clean[['ftbiden', 'fttrump', 'ftobama']]

,ftbiden,fttrump,ftobama
0,52.0,47.0,0.971498
1,41.0,41.0,-0.626842
2,88.0,0.0,0.998137
3,0.0,100.0,-0.094062
4,25.0,94.0,-0.946510
...,...,...,...
3160,7.0,81.0,-0.893232
3161,25.0,35.0,-0.333813
3162,50.0,6.0,-0.600203
3163,95.0,1.0,1.237888


Then we can reconstruct the original values of `ftobama`:

In [52]:
anes_clean = anes_clean.assign(ftobama = anes_clean.ftobama*obama_sd + obama_mean)
anes_clean[['ftbiden', 'fttrump', 'ftobama']]

,ftbiden,fttrump,ftobama
0,52.0,47.0,90.0
1,41.0,41.0,30.0
2,88.0,0.0,91.0
3,0.0,100.0,50.0
4,25.0,94.0,18.0
...,...,...,...
3160,7.0,81.0,20.0
3161,25.0,35.0,41.0
3162,50.0,6.0,31.0
3163,95.0,1.0,100.0


### Breaking Continuous Features into Categories
A useful function in `pandas` is `pd.cut()` which creates categories from break points in a continuous-valued column. There are three arguments for `pd.cut()`: first, the continuous column whose values we want to categorize, second `bins` takes either an integer to express the number of uniformly spaced categories, or a list of the breakpoints (inclusive for the upper bound but not the lower bound), and `labels` is a tuple of the labels to assign to each category. To cut the `ftbiden` column into three categories, we can write

In [53]:
anes_clean['ftbiden_level'] = pd.cut(anes_clean.ftbiden, bins=[-0.1,40,70,100], labels=("dislike", "neutral", "like"))
anes_clean[['ftbiden', 'ftbiden_level']]

,ftbiden,ftbiden_level
0,52.0,neutral
1,41.0,neutral
2,88.0,like
3,0.0,dislike
4,25.0,dislike
...,...,...
3160,7.0,dislike
3161,25.0,dislike
3162,50.0,neutral
3163,95.0,like


Or equivalently,

In [54]:
anes_clean = anes_clean.assign(ftbiden_level = 
                               pd.cut(anes_clean.ftbiden, 
                                      bins=[-0.1,40,70,100], 
                                      labels=("dislike", "neutral", "like")))
anes_clean[['ftbiden', 'ftbiden_level']]

,ftbiden,ftbiden_level
0,52.0,neutral
1,41.0,neutral
2,88.0,like
3,0.0,dislike
4,25.0,dislike
...,...,...
3160,7.0,dislike
3161,25.0,dislike
3162,50.0,neutral
3163,95.0,like


### Indexing Versus `.assign()`
The advantage of using `.assign()` as opposed to writing new columns into the dataframe's index is that we can create or replace multiple columns within one call to `.assign()` by separating each expression with commas. We can calculate `age` from `birthyr` and we can also generate a column containing the square of age (which is useful to place a curvilinear effect into a linear regression model): 

In [55]:
anes_clean = anes_clean.assign(age = 2020 - anes_clean.birthyr,
                              age2 = (2020 - anes_clean.birthyr)**2)
anes_clean[['vote', 'ftbiden', 'fttrump', 'age', 'age2']]

,vote,ftbiden,fttrump,age,age2
0,Joe Biden,52.0,47.0,51,2601
1,Donald Trump,41.0,41.0,78,6084
2,Joe Biden,88.0,0.0,66,4356
3,Donald Trump,0.0,100.0,41,1681
4,Donald Trump,25.0,94.0,80,6400
...,...,...,...,...,...
3160,Donald Trump,7.0,81.0,72,5184
3161,Someone else,25.0,35.0,24,576
3162,Probably will not vote,50.0,6.0,40,1600
3163,Joe Biden,95.0,1.0,60,3600


A warning: if you want to perform a calculation that uses a column you create, you must create the columns you will use first, then use them in a separate call to `.assign()`. Suppose that I had called `age` directly to create `age2` as follows:
```
anes_clean.assign(age = 2020 - anes_clean.birthyr,
                  age2 = anes_clean.age**2)
```
This code will not work because `age` is not yet defined until the end of the entire call to `.assign()`, so it cannot be used to create `age2`.

### Changing a Column's Data Type
One important way we might need to create a new column or revise an existing one is by **casting** the column to a different data type. Every column in the dataframe has a data type, which we can see listed with the `.dtypes` attribute. If we want to change the data type, we can use the `.astype()` method applied to the column whose type we want to change. To convert the column's values to strings type `.astype('str')`, to convert the column's values to integers type `.astype('int')`, and to convert the column's values to floats type `.astype('float')`. `pandas` also recognizes a categorical data type, and to convert a column to categorical type `.astype('category')`. As an example, let's convert the Joe Biden thermometer scores to each data type:

In [56]:
anes_clean = anes_clean.assign(ftbiden_float = anes_clean.ftbiden.astype('float'),
                              ftbiden_cat = anes_clean.ftbiden.astype('category'),
                              ftbiden_str = anes_clean.ftbiden.astype('str'))
anes_clean[['ftbiden', 'ftbiden_float', 'ftbiden_cat', 'ftbiden_str']].dtypes

ftbiden           float64
ftbiden_float     float64
ftbiden_cat      category
ftbiden_str        object
dtype: object

By default, missing values `np.nan` are stored as floats, so it is not possible to convert a column to integer if there are missing values.

We can convert all of the categorical columns in `anes_clean` to categorical data types as follows:

In [57]:
catcolumns = ['liveurban', 'vote16', 'protest', 'vote','confecon', 'ideology', 'partyID',
       'universal_income', 'family_separation', 'free_college','forgive_loans', 'race', 'sex', 'education']
anes_clean[catcolumns] = anes_clean[catcolumns].astype('category')
anes_clean.dtypes

caseid                     int64
liveurban               category
vote16                  category
protest                 category
vote                    category
most_important_issue      object
confecon                category
ideology                category
partyID                 category
universal_income        category
family_separation       category
free_college            category
forgive_loans           category
race                    category
birthyr                    int64
sex                     category
education               category
weight                    object
fttrump                  float64
ftobama                  float64
ftbiden                  float64
ftwarren                 float64
ftsanders                float64
ftbuttigieg              float64
ftharris                 float64
ftblack                  float64
ftwhite                  float64
fthisp                   float64
ftasian                  float64
ftmuslim                 float64
ftillegal 

### Using Logical Operators
Logical expressions can be used to create new columns. Columns that are set equal to a logical condition are populated by values that are either `True` when the logical condition is true, or `False` when the logical condition is false. To create a binary column indicating the participants who like Biden more than Trump, we can type

In [58]:
anes_clean['prefersbiden'] = anes_clean.ftbiden > anes_clean.fttrump
anes_clean[['vote','ftbiden', 'fttrump', 'prefersbiden']]

,vote,ftbiden,fttrump,prefersbiden
0,Joe Biden,52.0,47.0,True
1,Donald Trump,41.0,41.0,False
2,Joe Biden,88.0,0.0,True
3,Donald Trump,0.0,100.0,False
4,Donald Trump,25.0,94.0,False
...,...,...,...,...
3160,Donald Trump,7.0,81.0,False
3161,Someone else,25.0,35.0,False
3162,Probably will not vote,50.0,6.0,True
3163,Joe Biden,95.0,1.0,True


Or using `.assign()`:

In [59]:
anes_clean = anes_clean.assign(prefersbiden = anes_clean.ftbiden > anes_clean.fttrump)
anes_clean[['vote','ftbiden', 'fttrump', 'prefersbiden']]

,vote,ftbiden,fttrump,prefersbiden
0,Joe Biden,52.0,47.0,True
1,Donald Trump,41.0,41.0,False
2,Joe Biden,88.0,0.0,True
3,Donald Trump,0.0,100.0,False
4,Donald Trump,25.0,94.0,False
...,...,...,...,...
3160,Donald Trump,7.0,81.0,False
3161,Someone else,25.0,35.0,False
3162,Probably will not vote,50.0,6.0,True
3163,Joe Biden,95.0,1.0,True


Logical expressions can work in two ways: **elementwise** or **objectwise**. When a logical expression is evaluated elementwise, it matches the corresponding elements in multiple columns and returns a column of equal length that is populated by `True` and `False` values depending on the values on the same row. When a logical expression is evaluated objectwise, it considers whether a statement is true regarding a column as a whole. For example, the `==` sign asks whether the first object is equal to the second object. If I define two lists, `[1,2,3]` and `[3,2,1]` and evaluate them using `==`, I get:

In [60]:
[1,2,3] == [3,2,1]

False

The output is false because `[1,2,3] == [3,2,1]` is evaluated *objectwise*. The question Python answered was: is the list `[1,2,3]` the same as the list `[3,2,1]`? These two lists are not the same, so the answer is `False`. If we convert these two lists to `numpy` arrays instead, then the same comparison is *elementwise*:

In [61]:
np.array([1,2,3]) == np.array([3,2,1])

array([False,  True, False])

In this case, we compare corresponding elements and find that the first elements are not the same, the second elements are the same, and the third elements are not the same, so the answer is a list: `[False, True, False]`. Because `pandas` dataframes are built using `numpy` under the hood, individual columns are `numpy` arrays, and some logical operators - namely `<`, `<=`, `>`, `>=`, `~`, `&`, and `|` - work elementwise. That's why the commands listed above to generate Boolean features in `anes_clean` work. 

However, some operators do not work elementwise, even within `numpy` arrays. Python includes two versions of "and", for example: in addition to `&`, there is also `and`. While `&` works elementwise, `and` only works objectwise. For example the following expression

In [62]:
(np.array([1,2,3,4]) >= np.array([4,3,2,1])) & (np.array([1,2,3,4]) == np.array([5,4,3,2]))

array([False, False,  True, False])

yields an array of logical values, but switching the "and" operator:
```
(np.array([1,2,3,4]) >= np.array([4,3,2,1])) and (np.array([1,2,3,4]) == np.array([5,4,3,2]))
```
yields an error: "The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()". The only really important logical operator we will want to use that does not work elementwise in `pandas` is `in`. To create a new column that uses an objectwise operator like `in`, we have to loop across the rows of the dataframe using the `.apply()` method and a `lambda` function. `lambda` defines the operation that will be evaluated on each row, and constructs the sequence of logical values one row at a time. Using a loop is computationally inefficient, but it gets the job done. 

For example, if we want to create the `worried_econ` column using `in`, we can use the following code:

In [63]:
anes_clean['worried_econ'] = anes_clean.confecon.apply(
        lambda x: x in ["Moderately worried","Extremely worried"]
    )
anes_clean[['confecon', 'worried_econ']]    

,confecon,worried_econ
0,A little worried,False
1,A little worried,False
2,Extremely worried,True
3,A little worried,False
4,Not at all worried,False
...,...,...
3160,A little worried,False
3161,Extremely worried,True
3162,Very worried,False
3163,Moderately worried,True


The first line creates the `worried_econ` column and sets it equal to the output of the loop that is applied to the rows of `confecon`. Inside the loop, `lambda x` defines `x` as an index that represents each individual row. The function `x in ["Moderately worried","Extremely worried"]` returns `True` if the value of `confecon` on a particular row is "Moderately worried" or "Extremely worried" and `False` otherwise.

As another example, let's create a column that identifies the voters that favor a universal basic income, at least a little, and also favor free college tuition. In this case, we have to specify that `axis=1` so that the loop works down columns and applies the function on every row in the dataframe. The following code works:

In [64]:
anes_clean['favor_both'] = anes_clean.apply(
    lambda x: x['universal_income'] in ["Favor a little","Favor a moderate amount","Favor a great deal"] and
    x['free_college'] in ["Favor a little","Favor a moderate amount","Favor a great deal"],
    axis=1
)
anes_clean[['universal_income', 'free_college', 'favor_both']]

,universal_income,free_college,favor_both
0,Favor a moderate amount,Favor a moderate amount,True
1,Oppose a moderate amount,Oppose a little,False
2,Neither favor nor oppose,Oppose a moderate amount,False
3,Neither favor nor oppose,Favor a little,False
4,Oppose a great deal,Oppose a great deal,False
...,...,...,...
3160,Oppose a great deal,Oppose a great deal,False
3161,Favor a great deal,Favor a little,True
3162,Oppose a great deal,Oppose a great deal,False
3163,Favor a moderate amount,Neither favor nor oppose,False


## Data Aggregation
Data aggregation is the process of collapsing a dataframe to have one row for every group of a given categorical column, where other columns are summarized with statistics like the sum, mean, and median of values within each group. In `pandas`, data aggregation follows a **split-apply-combine** approach. First, the data are split into subsets for every group in the grouping column. Then a function, such as the mean, is applied to the same columns in each subset and each subset is compressed to have just one row. Then the subsets are combined into the final resulting dataframe.

The general syntax for aggregating a dataframe is
```
df.groupby('groupcolumn').aggfunctions.sort_values()
```

Commands to aggregate dataframes can get confusing very quickly. It helps to break down the syntax for data aggregation into parts for each method listed in the syntax. First, `df` is the name of the dataframe we want to compress. Second, `.groupby('groupcolumn')` specifies the column whose categories will define the rows of the aggregated dataframe. If more than one column defines these groups (examples below), then the syntax becomes `.groupby(['groupcolumn1', 'groupcolumn2'])`. Third, `.aggfunctions` defines how other columns in the data can be included and summarized in the aggregated data. Finally, `.sort_values()` is optional, but often helps to generate a better presentation for the aggregated data table.

As an example, let's collapse the ANES data by the candidates that people report they will vote for. We start with `anes_clean` and set the grouping variable with `.groupby('vote')`. We apply the `.size()` function, which reports the number of rows in each group, and we use `.sort_values(ascending=False)` to list these counts from biggest to smallest (no `by` argument is needed because there is only one column in the aggregated data). In all, this command provides a tally of people's voting intentions as of December 2019:


In [65]:
anes_clean.groupby('vote').size().sort_values(ascending=False)

vote
Joe Biden                 1288
Donald Trump              1273
Someone else               321
Probably will not vote     283
dtype: int64

So according to the ANES data, the election is close, but Joe Biden has the most votes in the sample.

To use aggregation functions other than `.size()`, the best option is to use the `.agg()` method instead. `.agg()` takes a dictionary as its argument, and this dictionary should contain the names of the columns we want to summarize, as keys, and the functions we want to use to collapse these columns, as values. For example, we can construct a table in which the rows are the voting groups, and the columns contain the mean feeling thermometers for Trump, Biden, immigrants, and journalists:

In [66]:
anes_clean.groupby('vote').agg({'fttrump':'mean',
                               'ftbiden':'mean',
                               'ftimmig':'mean',
                               'ftjournal':'mean'})

,fttrump,ftbiden,ftimmig,ftjournal
vote,,,,
Donald Trump,87.835303,15.743286,68.801887,26.502749
Joe Biden,8.450549,70.724649,75.315217,72.365683
Probably will not vote,28.750000,39.595238,61.734043,49.706714
Someone else,23.126582,34.022293,67.503125,52.386293


There are aggregation functions other than `'mean'` available: `count` gives the number of nonmissing values within the group, `sum` takes the sum of the values in each group, `min` and `max` report the minimum and maximum values, `median` reports the 50th percentile, `std` and `var` calculate the standard deviation and variance, `sem` takes the standard error of the mean, and `first` and `last` report the first and last value that appear in the subset defined by each group. For example, we can report all of these statistics for `ftbiden` across voting groups. If we want more than one statistic for a column, we can specify all of these functions in a list within the dictionary as follows:

In [67]:
anes_clean.groupby('vote').agg({'ftbiden':['count','mean','sum','min','max','median','std','var','sem','first','last']})

ftbiden                                         \
                         count       mean      sum  min    max median   
vote                                                                    
Donald Trump              1266  15.743286  19931.0  0.0  100.0    6.0   
Joe Biden                 1282  70.724649  90669.0  0.0  100.0   73.0   
Probably will not vote     252  39.595238   9978.0  0.0  100.0   48.0   
Someone else               314  34.022293  10683.0  0.0  100.0   32.0   

                                                                     
                              std         var       sem first  last  
vote                                                                 
Donald Trump            20.644470  426.194125  0.580212  41.0   7.0  
Joe Biden               22.384124  501.049025  0.625167  52.0  70.0  
Probably will not vote  25.337303  641.978941  1.596100  29.0  50.0  
Someone else            24.396672  595.197585  1.376784  20.0  25.0

More than one column may define the groups. For example, we might want to count the number of voters for each candidate in cities, in rural areas, in suburbs, and in towns. In that case, we can specify `groupby(['vote','liveurban'])` and `.size()` to count the number of voters of each type:

In [68]:
anes_clean.groupby(['vote','liveurban']).size()

vote                    liveurban
Donald Trump            City         235
                        Rural        314
                        Suburb       465
                        Town         259
Joe Biden               City         401
                        Rural        204
                        Suburb       462
                        Town         221
Probably will not vote  City          88
                        Rural         61
                        Suburb        83
                        Town          51
Someone else            City          92
                        Rural         64
                        Suburb       107
                        Town          58
dtype: int64

This table looks nicer as a `pandas` dataframe:

In [69]:
pd.DataFrame(anes_clean.groupby(['vote','liveurban']).size())

0
vote                   liveurban     
Donald Trump           City       235
                       Rural      314
                       Suburb     465
                       Town       259
Joe Biden              City       401
                       Rural      204
                       Suburb     462
                       Town       221
Probably will not vote City        88
                       Rural       61
                       Suburb      83
                       Town        51
Someone else           City        92
                       Rural       64
                       Suburb     107
                       Town        58

The table looks even nicer if we also apply the `.unstack()` method to move the data into different columns:

In [70]:
pd.DataFrame(anes_clean.groupby(['vote','liveurban']).size().unstack())

liveurban,City,Rural,Suburb,Town
vote,,,,
Donald Trump,235,314,465,259
Joe Biden,401,204,462,221
Probably will not vote,88,61,83,51
Someone else,92,64,107,58


Biden has more voters than Trump in cities, and Trump has more in rural areas. Trump has a slight lead among people in towns, and Trump and Biden are within three votes among people in suburbs. It looks like the election will depend on voters from the suburbs.

The standard functions in `.agg()` will usually suffice, but there are situations in which we might want to use a custom aggregation function. For that, we can use `.apply()` and a `lambda` function. The following code searches the `most_important_issue` column, which is a free text entry field in which respondents wrote their answer for the most important issue in the country today, and counts the number of times the word "trump" is mentioned in these responses:

In [71]:
anes_clean.groupby('vote').most_important_issue.apply(
    lambda x: x.str.contains("trump", case=False).sum()
).sort_values(ascending=False)

vote
Joe Biden                 325
Donald Trump               59
Someone else               25
Probably will not vote     10
Name: most_important_issue, dtype: int64

Biden voters mentioned "trump" 325 times, and Trump voters only mentioned "trump" 59 times. This code groups by voting group, then selects the `most_important_issue` column and uses the `.apply()` method on this column which creates a loop across groups. Within `.apply()`, the `lambda` function denotes a token `x` that represents the `most_important_issue` column within each group. On this column, the function uses the `.str.contains("trump", case=False)` function, which outputs `True` if the string "trump" is found within each value of `most_important_issue` without case sensitivity, and `False` otherwise. Finally,the `.sum()` function counts the number of times these searches were `True`.

Finally, we can save the cleaned ANES data as a separate CSV:

In [72]:
anes_clean.to_csv('anes_pilot2019_clean.csv', index=False)